In [1]:
#import required packages
import os
import ast
import urllib2
import urllib
import pandas as pd
import json
from cStringIO import StringIO
import scipy.stats
from scipy.cluster.vq import whiten
from matplotlib import gridspec
from sklearn.cluster import KMeans
import statsmodels.formula.api as smf
from pandas.tools.plotting import scatter_matrix
import geopandas as gpd
import glob
from fiona.crs import from_epsg
import datetime as dt
import zipfile
from matplotlib import cm
from descartes import PolygonPatch
import scipy.cluster as scp
import pylab as pl
from pandas.io.common import ZipFile
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from geopandas import GeoDataFrame
import seaborn as sns
from sklearn import cluster
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## Task1: Calculate travel times of the complete route between the following Stop IDs:
- 553332 ROOSEVELT AV/61 ST NB
- 551576 74 ST/ROOSEVELT AV STATION NB
- 804110 LaGUARDIA INTERNAL RDWY/TERMINAL B NB
- 503875 LGA/TERMINAL D SB
- 503848 LGA/TERMINAL C SB
- 505192 ROOSEVELT AV/74 ST SB
- 982075 61 ST/ROOSEVELT AV SB

In [2]:
#read in data
mta = pd.read_csv('all-records_2017-12-10.csv')
mta

id        lat        lon     bearing  \
0      d254a85a-58dc-4d31-bdaf-ba343654cad9  40.801521 -73.968296  338.482880   
1      c5321b0c-a245-4a3a-aa94-4a177a917222  40.801521 -73.968296  338.482880   
2      7c125d31-3517-47d0-8e5c-b0ce1b2f052c  40.801521 -73.968296  338.482880   
3      6fba1c52-f3ec-499e-a82d-9e5362571030  40.801521 -73.968296  338.482880   
4      89586119-c515-428c-b851-46f8a4cff420  40.801750 -73.968527  234.762400   
5      cc7d4b62-d681-4340-88d0-c63dbbd71eb2  40.803815 -73.967087  233.865600   
6      6f666d61-4b2f-43da-8a09-9d49fae041d0  40.803815 -73.967087  233.865600   
7      b87d81fc-0064-4254-a382-0ca4f5bf2b44  40.804821 -73.966355  233.977810   
8      6e1a0dbb-6ad5-40a1-8149-dedd3cb63bf6  40.805036 -73.966197  233.690640   
9      affa6e56-cdb8-442b-8259-9024853e7c4f  40.805825 -73.965620  234.075840   
10     99c14179-41a4-49e3-9693-bb00d0975eba  40.807872 -73.964129  233.901720   
11     1e5fb3a8-31ff-4914-a1e3-67e0dc15cc42  40.810071 -73.962524  233.746150   
12     be75d0e0-66b4-4955-939f-d6e54f2c52df  40.810554 -73.962109  158.821910   
13     e84c721e-6583-473c-ad04-b919b5c9840f  40.809353 -73.959205  234.016050   
14     aa4cda23-f1ef-40cd-9096-366878bcdecf  40.809353 -73.959205  234.016050   
15     df4ebd4b-3736-406c-877e-daefcbdd718d  40.809353 -73.959205  234.016050   
16     76103f1f-d4c1-47e7-b142-42a41b26e36d  40.811195 -73.957861  233.988680   
17     13968313-ada3-4f0d-81c0-abf3f7427d04  40.812425 -73.956963  233.851410   
18     91a4483d-48ad-4c34-84bb-84d726c6dd5e  40.813356 -73.956254  130.973820   
19     f4b8d4d3-9a05-44fa-8ecd-0ec4b4569ffb  40.811890 -73.954980  130.973820   
20     8fd2361f-9f22-4aee-a73d-c64f1d3ae226  40.810600 -73.952198  157.481920   
21     527e7a70-cabd-4919-ac8e-8ef34b3973d3  40.810208 -73.951252  157.502470   
22     a9800c84-934c-4f00-a992-fe1de104c74f  40.809093 -73.948616  157.208070   
23     df841184-dbae-4568-b490-ce53c787d1db  40.808031 -73.946129  157.796520   
24     3cdd34b5-0ec4-437b-af7a-d3ef95e0ddb3  40.808031 -73.946129  157.796520   
25     48efda14-6029-4cfb-91aa-6e51350a60e5  40.807641 -73.945173  157.148500   
26     6d2f9dac-2975-4bcc-944b-7bfe74e383c4  40.807661 -73.945220  157.148500   
27     3aac0af2-04a4-48e6-87df-b354382443e6  40.806759 -73.943081  157.148500   
28     ddc2ec61-f3bb-4831-afe9-37ce9c076bd2  40.806219 -73.941797  157.221820   
29     d9c59b1e-9804-4680-b681-98d15eda8ab5  40.805062 -73.939052  157.054260   
...                                     ...        ...        ...         ...   
34583  1d7caaee-7ad7-4e14-87af-f67400d0cfac  40.771100 -73.877468   18.392912   
34584  a90f781b-026a-4c29-8f86-13a522f12d4a  40.768362 -73.883005   24.659197   
34585  d15032c9-6c15-42dc-9ff3-8e6eb7b5ba67  40.766477 -73.889234    8.517230   
34586  a86f3f09-322b-41a0-a436-67519f41b39b  40.765023 -73.894718   68.903694   
34587  2543382a-6201-42ea-81c1-b18aa1be4d9d  40.761262 -73.897663   56.309933   
34588  7fd1bba1-46d9-4f10-b559-80c82ae1fcba  40.756526 -73.899556  109.374140   
34589  def31aa2-0259-44c3-936c-38ec48f1a3f8  40.751945 -73.897330  104.282585   
34590  2cc289a2-f9e6-4caa-99a3-f98a50ac6f23  40.748551 -73.895741  159.443950   
34591  880c13e5-eaec-41db-b696-ae4b43373795  40.748425 -73.895447  155.835700   
34592  cdf0c7f3-454b-4eb3-addd-0f9b887362a8  40.748133 -73.894762  157.437940   
34593  5e1e169a-2a1d-4253-9bc5-449e2607cbb4  40.748133 -73.894762  157.437940   
34594  2d1793cc-a5a4-448b-a47c-bc763b0859e0  40.747306 -73.892798  157.607940   
34595  0b8b371e-ef53-4a09-b6fb-b9c7c948e232  40.746660 -73.891345  157.174970   
34596  0a50e5ea-6361-4f2c-ad39-5858f42e5465  40.746660 -73.891345  157.174970   
34597  d2b69168-0b6a-44ef-ace1-d88ab880cdb0  40.746703 -73.891442  155.389160   
34598  c9fc8849-d627-4193-8802-b206d1da9c4a  40.746584 -73.890537  211.827450   
34599  3d4fde8a-3e62-4b64-9f5b-2b478faf6bfb  40.746942 -73.890816    6.044894   
34600  988001f9-8f12-4129-b79e-e6e06f3a3e7e  40.746896 -73.891251    6.0

In [3]:
#check route
mta.route.unique()

array(['MTA NYCT_M60+', 'MTABC_Q70+'], dtype=object)

In [4]:
#selecting only entries in route Q70+
df = mta.loc[mta['route'] == 'MTABC_Q70+']

In [5]:
df.head()

id        lat        lon    bearing  \
714  004d09d3-b663-4886-88ac-e040057b39cc  40.746021 -73.903194  243.43495   
720  8cf5b06a-1322-49e2-ada0-323f952a29f0  40.746100 -73.903158  247.83365   
726  cb1e5f7b-7659-42e3-afca-5661f119024c  40.747111 -73.902236  164.99403   
731  c204bdfd-6a00-466c-86f3-e34a0457df03  40.746678 -73.900569  165.79782   
736  337c2c9e-488a-46cd-a809-f55bcbc99e9b  40.746203 -73.898689  168.09782   

          route  direction service_date                          gtfs_trip_id  \
714  MTABC_Q70+          0   2017-12-10  MTABC_18522877-LGPE7-LG_E7-Sunday-70   
720  MTABC_Q70+          0   2017-12-10  MTABC_18522877-LGPE7-LG_E7-Sunday-70   
726  MTABC_Q70+          0   2017-12-10  MTABC_18522877-LGPE7-LG_E7-Sunday-70   
731  MTABC_Q70+          0   2017-12-10  MTABC_18522877-LGPE7-LG_E7-Sunday-70   
736  MTABC_Q70+          0   2017-12-10  MTABC_18522877-LGPE7-LG_E7-Sunday-70   

     gtfs_shape_id gtfs_short_route_name       ...       sched_arrive  \
714  MTA_SBS700016               Q70-SBS       ...                NaN   
720  MTA_SBS700016               Q70-SBS       ...                NaN   
726  MTA_SBS700016               Q70-SBS       ...                NaN   
731  MTA_SBS700016               Q70-SBS       ...                NaN   
736  MTA_SBS700016               Q70-SBS       ...                NaN   

    next_predicted_arrival_time next_predicted_departure_time  \
714  2017-12-11 00:24:32.304000    2017-12-11 00:24:32.304000   
720  2017-12-11 00:24:00.716000    2017-12-11 00:24:00.716000   
726  2017-12-11 00:24:22.725000    2017-12-11 00:24:22.725000   
731  2017-12-11 00:24:35.056000    2017-12-11 00:24:35.056000   
736  2017-12-11 00:24:52.928000    2017-12-11 00:24:52.928000   

    next_stop_distance_from_origin  next_stop_distance_to_vehicle  \
714                       15252.51                          32.47   
720                       15252.51                          41.75   
726                       15252.51                         213.37   
731                       15252.51                         361.80   
736                       15252.51                         528.78   

     num_stops_from_call stops_from_call_presentable     recorded_at_time  \
714                    0                 approaching  2017-12-11 00:24:22   
720                    0                 approaching  2017-12-11 00:23:50   
726                    0               < 1 stop away  2017-12-11 00:23:19   
731                    0               < 1 stop away  2017-12-11 00:22:47   
736                    0               < 1 stop away  2017-12-11 00:22:15   

    stop_distance_difference at_stop_boolean  
714                      NaN               0  
720                      NaN               0  
726                      NaN               0  
731                      NaN               0  
736                      NaN               0  

[5 rows x 31 columns]

In [6]:
#check direction 
df.direction.unique()

array([0])

In [7]:
df.columns

Index([u'id', u'lat', u'lon', u'bearing', u'route', u'direction',
       u'service_date', u'gtfs_trip_id', u'gtfs_shape_id',
       u'gtfs_short_route_name', u'gtfs_agency_id', u'origin_stop_id',
       u'destination_stop_id', u'gtfs_trip_headsign',
       u'origin_aimed_departure_time', u'situation_ref', u'progress_rate',
       u'progress_status', u'vehicle_id', u'gtfs_block_id', u'next_stop_id',
       u'sched_arrive', u'next_predicted_arrival_time',
       u'next_predicted_departure_time', u'next_stop_distance_from_origin',
       u'next_stop_distance_to_vehicle', u'num_stops_from_call',
       u'stops_from_call_presentable', u'recorded_at_time',
       u'stop_distance_difference', u'at_stop_boolean'],
      dtype='object')

In [8]:
#check stops
df.origin_stop_id.unique()

array(['MTA_553332'], dtype=object)

In [9]:
df.destination_stop_id.unique()

array(['MTA_982075'], dtype=object)

In [10]:
df.next_stop_id.unique()

array(['MTA_982075', 'MTA_505192', 'MTA_804110', 'MTA_503875',
       'MTA_503848', 'MTA_551576', 'MTA_553332'], dtype=object)

In [11]:
df['recorded_at_time'] = pd.to_datetime(df['recorded_at_time'])
df['next_predicted_arrival_time'] = pd.to_datetime(df['next_predicted_arrival_time'])

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [12]:
df.sort_values(['recorded_at_time'],ascending=True, inplace=True)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
df

id        lat        lon     bearing  \
34612  f958a26b-b017-4654-ae01-c23851c65a33  40.745688 -73.902816    4.899093   
34611  c9720755-0d32-48ed-9eb2-cfcb68d79ec3  40.745688 -73.902816    4.899093   
34610  f851c396-afdb-4ef8-b37b-93f7bbde6af2  40.745688 -73.902816    4.899093   
34609  5a2bdbe4-c4eb-4552-8230-bdfac12ba553  40.745688 -73.902816    4.899093   
34608  eca20a41-4761-4b58-b6cc-ea7453f01631  40.745688 -73.902816    4.899093   
34607  8b054d07-c9c8-4187-8570-00a02d5ea894  40.745688 -73.902816    4.899093   
34606  b2bf1cfa-5928-4629-9d21-ca9be61c015d  40.745716 -73.902555    6.472271   
34605  d7d3f310-b135-458a-9ccf-ad67fa21711e  40.746040 -73.899424    6.217817   
34604  555023ca-2865-4ddf-a22f-cf94d63f36b0  40.746040 -73.899424    6.217817   
34603  0468b507-5e38-4bce-9fc8-4e0e0026317c  40.746228 -73.897642    5.697729   
34602  54b3cd5b-9e93-4573-b305-caacff62c628  40.746599 -73.894121    6.110895   
34601  f3669770-46c3-40f8-96ce-029b3a6d0503  40.746764 -73.892556    5.932704   
34600  988001f9-8f12-4129-b79e-e6e06f3a3e7e  40.746896 -73.891251    6.044894   
34599  3d4fde8a-3e62-4b64-9f5b-2b478faf6bfb  40.746942 -73.890816    6.044894   
34598  c9fc8849-d627-4193-8802-b206d1da9c4a  40.746584 -73.890537  211.827450   
34597  d2b69168-0b6a-44ef-ace1-d88ab880cdb0  40.746703 -73.891442  155.389160   
34596  0a50e5ea-6361-4f2c-ad39-5858f42e5465  40.746660 -73.891345  157.174970   
34595  0b8b371e-ef53-4a09-b6fb-b9c7c948e232  40.746660 -73.891345  157.174970   
34594  2d1793cc-a5a4-448b-a47c-bc763b0859e0  40.747306 -73.892798  157.607940   
34593  5e1e169a-2a1d-4253-9bc5-449e2607cbb4  40.748133 -73.894762  157.437940   
34592  cdf0c7f3-454b-4eb3-addd-0f9b887362a8  40.748133 -73.894762  157.437940   
34591  880c13e5-eaec-41db-b696-ae4b43373795  40.748425 -73.895447  155.835700   
34590  2cc289a2-f9e6-4caa-99a3-f98a50ac6f23  40.748551 -73.895741  159.443950   
34589  def31aa2-0259-44c3-936c-38ec48f1a3f8  40.751945 -73.897330  104.282585   
34588  7fd1bba1-46d9-4f10-b559-80c82ae1fcba  40.756526 -73.899556  109.374140   
34587  2543382a-6201-42ea-81c1-b18aa1be4d9d  40.761262 -73.897663   56.309933   
34586  a86f3f09-322b-41a0-a436-67519f41b39b  40.765023 -73.894718   68.903694   
34585  d15032c9-6c15-42dc-9ff3-8e6eb7b5ba67  40.766477 -73.889234    8.517230   
34584  a90f781b-026a-4c29-8f86-13a522f12d4a  40.768362 -73.883005   24.659197   
34583  1d7caaee-7ad7-4e14-87af-f67400d0cfac  40.771100 -73.877468   18.392912   
...                                     ...        ...        ...         ...   
898    a50f78b5-b86c-4ffa-bc8d-79c8ee2bf02f  40.746776 -73.902875  247.319440   
890    77e3bf4d-83c2-412f-87e2-9b25719252e5  40.745786 -73.903307  244.368410   
889    3fbe8c8a-df23-4856-9d7a-2fe0edcdcb20  40.746887 -73.891339    6.410840   
882    cd5c2278-eefd-41bf-90fa-ef080edf7d7b  40.745786 -73.903307  244.368410   
881    492c7eb2-007d-4e59-9603-decc4fa02061  40.746905 -73.891162    6.079589   
875    227d8ed4-7121-4102-8ab1-a3dd99792164  40.745786 -73.903307  244.368410   
874    03535ac7-ecd7-4202-8d22-27781ddd1087  40.746905 -73.891162    6.079589   
867    610540d6-2a97-43a8-8548-90f92178ff18  40.745786 -73.903307  244.368410   
868    1aa51860-5574-4284-8e3c-5edacf3b0f8b  40.746894 -73.891271    5.893529   
861    5424bd98-458c-4a5e-aa95-88b14e25d43f  40.745786 -73.903307  244.368410   
852    34c99b0b-b6fa-4ab6-8e5d-fa1fa0cc73ca  40.746952 -73.890726    6.079589   
860    720baddc-b892-44aa-80cf-ca382ce40e25  40.746952 -73.890726    6.079589   
844    74506f88-414b-4c6c-b499-d38fe8ed78d4  40.745786 -73.903307  244.368410   
853    b104255d-cae7-4e5e-a6b2-41d5ac59e31a  40.745786 -73.903307  244.368410   
843    572306b8-ccc6-47a8-84ad-b4dd4b41bfec  40.746441 -73.890424  242.447190   
835    a6a1466c-6d01-4307-99b3-9611e7b3068e  40.745786 -73.903307  244.368410   
834    9b0bcdb7-736f-4880-9b8f-e4c4f9d2ae36  40.746441 -73.890424  242.447190   
828    21e4f85d-d2ba-485d-bbb5-ff00fbaedbd4  40.745786 -73.903307  244.3

In [14]:
df.vehicle_id.unique()

array(['MTABC_7451', 'MTABC_7449', 'MTABC_7448', 'MTABC_7450',
       'MTABC_7453', 'MTABC_7446', 'MTABC_7445', 'MTABC_7472',
       'MTABC_7455', 'MTABC_7463'], dtype=object)

There are 10 buses running the same route

In [15]:
df.columns

Index([u'id', u'lat', u'lon', u'bearing', u'route', u'direction',
       u'service_date', u'gtfs_trip_id', u'gtfs_shape_id',
       u'gtfs_short_route_name', u'gtfs_agency_id', u'origin_stop_id',
       u'destination_stop_id', u'gtfs_trip_headsign',
       u'origin_aimed_departure_time', u'situation_ref', u'progress_rate',
       u'progress_status', u'vehicle_id', u'gtfs_block_id', u'next_stop_id',
       u'sched_arrive', u'next_predicted_arrival_time',
       u'next_predicted_departure_time', u'next_stop_distance_from_origin',
       u'next_stop_distance_to_vehicle', u'num_stops_from_call',
       u'stops_from_call_presentable', u'recorded_at_time',
       u'stop_distance_difference', u'at_stop_boolean'],
      dtype='object')

In [16]:
#select only required columns
df1 = df[['id', 'lat', 'lon', 'vehicle_id', 'next_stop_id','next_predicted_arrival_time', 'next_predicted_departure_time','next_stop_distance_from_origin', 'recorded_at_time', 'at_stop_boolean']]
df1.head(20)

id        lat        lon  vehicle_id  \
34612  f958a26b-b017-4654-ae01-c23851c65a33  40.745688 -73.902816  MTABC_7451   
34611  c9720755-0d32-48ed-9eb2-cfcb68d79ec3  40.745688 -73.902816  MTABC_7451   
34610  f851c396-afdb-4ef8-b37b-93f7bbde6af2  40.745688 -73.902816  MTABC_7451   
34609  5a2bdbe4-c4eb-4552-8230-bdfac12ba553  40.745688 -73.902816  MTABC_7451   
34608  eca20a41-4761-4b58-b6cc-ea7453f01631  40.745688 -73.902816  MTABC_7451   
34607  8b054d07-c9c8-4187-8570-00a02d5ea894  40.745688 -73.902816  MTABC_7451   
34606  b2bf1cfa-5928-4629-9d21-ca9be61c015d  40.745716 -73.902555  MTABC_7451   
34605  d7d3f310-b135-458a-9ccf-ad67fa21711e  40.746040 -73.899424  MTABC_7451   
34604  555023ca-2865-4ddf-a22f-cf94d63f36b0  40.746040 -73.899424  MTABC_7451   
34603  0468b507-5e38-4bce-9fc8-4e0e0026317c  40.746228 -73.897642  MTABC_7451   
34602  54b3cd5b-9e93-4573-b305-caacff62c628  40.746599 -73.894121  MTABC_7451   
34601  f3669770-46c3-40f8-96ce-029b3a6d0503  40.746764 -73.892556  MTABC_7451   
34600  988001f9-8f12-4129-b79e-e6e06f3a3e7e  40.746896 -73.891251  MTABC_7451   
34599  3d4fde8a-3e62-4b64-9f5b-2b478faf6bfb  40.746942 -73.890816  MTABC_7451   
34598  c9fc8849-d627-4193-8802-b206d1da9c4a  40.746584 -73.890537  MTABC_7451   
34597  d2b69168-0b6a-44ef-ace1-d88ab880cdb0  40.746703 -73.891442  MTABC_7451   
34596  0a50e5ea-6361-4f2c-ad39-5858f42e5465  40.746660 -73.891345  MTABC_7451   
34595  0b8b371e-ef53-4a09-b6fb-b9c7c948e232  40.746660 -73.891345  MTABC_7451   
34594  2d1793cc-a5a4-448b-a47c-bc763b0859e0  40.747306 -73.892798  MTABC_7451   
34593  5e1e169a-2a1d-4253-9bc5-449e2607cbb4  40.748133 -73.894762  MTABC_7451   

      next_stop_id next_predicted_arrival_time next_predicted_departure_time  \
34612   MTA_553332                         NaT                           NaN   
34611   MTA_553332                         NaT                           NaN   
34610   MTA_553332                         NaT                           NaN   
34609   MTA_553332                         NaT                           NaN   
34608   MTA_553332                         NaT                           NaN   
34607   MTA_553332                         NaT                           NaN   
34606   MTA_551576     2017-12-10 00:04:36.835    2017-12-10 00:04:36.835000   
34605   MTA_551576     2017-12-10 00:04:11.304    2017-12-10 00:04:11.304000   
34604   MTA_551576     2017-12-10 00:04:43.304    2017-12-10 00:04:43.304000   
34603   MTA_551576     2017-12-10 00:04:44.112    2017-12-10 00:04:44.112000   
34602   MTA_551576     2017-12-10 00:04:11.515    2017-12-10 00:04:11.515000   
34601   MTA_551576     2017-12-10 00:04:16.249    2017-12-10 00:04:16.249000   
34600   MTA_551576     2017-12-10 00:04:24.695    2017-12-10 00:04:24.695000   
34599   MTA_551576     2017-12-10 00:04:47.843    2017-12-10 00:04:47.843000   
34598   MTA_551576     2017-12-10 00:05:15.884    2017-12-10 00:05:15.884000   
34597   MTA_804110     2017-12-10 00:14:24.912    2017-12-10 00:14:24.912000   
34596   MTA_804110     2017-12-10 00:14:57.908    2017-12-10 00:14:57.908000   
34595   MTA_804110     2017-12-10 00:15:28.908    2017-12-10 00:15:28.908000   
34594   MTA_804110     2017-12-10 00:15:46.967    2017-12-10 00:15:46.967000   
34593   MTA_804110     2017-12-10 00:15:58.045    2017-12-10 00:15:58.045000   

       next_stop_distance_from_origin    recorded_at_time  at_stop_boolean  
34612                            0.01 2017-12-09 23:58:07                1  
34611                            0.01 2017-12-09 23:58:40                1  
34610                            0.01 2017-12-09 23:59:11                1  
34609                            0.01 2017-12-09 23:59:42                1  
34608                            0.01 2017-12-09 23:59:42                1  
34607                            0.01 2017-12-10 00:00:14                1  
34606                         1109.27 2017-12-10 00:00:46                0  
34605                         1109.27 2017

In [17]:
df1['time'] = df1['recorded_at_time'].dt.time

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
df1['time'] = df1['time'].astype(str)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [19]:
df1['time'] = df1['time'].str.split(':').str[0]
df1['time'] = df1['time'].astype(int)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Sort times into intervals:

- 0-5
- 6-11
- 12-17
- 18-23

In [20]:
# sorting into time interval bins
df1['TIMEINT'] = pd.cut(df1['time'], bins=[0,5,12,18,300], labels=[1,2,3,4])
df1.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
34612  f958a26b-b017-4654-ae01-c23851c65a33  40.745688 -73.902816  MTABC_7451   
34611  c9720755-0d32-48ed-9eb2-cfcb68d79ec3  40.745688 -73.902816  MTABC_7451   
34610  f851c396-afdb-4ef8-b37b-93f7bbde6af2  40.745688 -73.902816  MTABC_7451   
34609  5a2bdbe4-c4eb-4552-8230-bdfac12ba553  40.745688 -73.902816  MTABC_7451   
34608  eca20a41-4761-4b58-b6cc-ea7453f01631  40.745688 -73.902816  MTABC_7451   

      next_stop_id next_predicted_arrival_time next_predicted_departure_time  \
34612   MTA_553332                         NaT                           NaN   
34611   MTA_553332                         NaT                           NaN   
34610   MTA_553332                         NaT                           NaN   
34609   MTA_553332                         NaT                           NaN   
34608   MTA_553332                         NaT                           NaN   

       next_stop_distance_from_origin    recorded_at_time  at_stop_boolean  \
34612                            0.01 2017-12-09 23:58:07                1   
34611                            0.01 2017-12-09 23:58:40                1   
34610                            0.01 2017-12-09 23:59:11                1   
34609                            0.01 2017-12-09 23:59:42                1   
34608                            0.01 2017-12-09 23:59:42                1   

       time TIMEINT  
34612    23       4  
34611    23       4  
34610    23       4  
34609    23       4  
34608    23       4

In [21]:
#df1[df1['TIMEINT'].isnull()]

In [22]:
df1['TIMEINT'].fillna(1, inplace=True)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/pandas/core/generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [23]:
df1 = df1.loc[df1['at_stop_boolean'] == 1]

In [24]:
df1

id        lat        lon  vehicle_id  \
34612  f958a26b-b017-4654-ae01-c23851c65a33  40.745688 -73.902816  MTABC_7451   
34611  c9720755-0d32-48ed-9eb2-cfcb68d79ec3  40.745688 -73.902816  MTABC_7451   
34610  f851c396-afdb-4ef8-b37b-93f7bbde6af2  40.745688 -73.902816  MTABC_7451   
34609  5a2bdbe4-c4eb-4552-8230-bdfac12ba553  40.745688 -73.902816  MTABC_7451   
34608  eca20a41-4761-4b58-b6cc-ea7453f01631  40.745688 -73.902816  MTABC_7451   
34607  8b054d07-c9c8-4187-8570-00a02d5ea894  40.745688 -73.902816  MTABC_7451   
34598  c9fc8849-d627-4193-8802-b206d1da9c4a  40.746584 -73.890537  MTABC_7451   
34465  69a9f1a3-c74b-4c84-b183-80045d5c51e1  40.770190 -73.864224  MTABC_7451   
34464  d25bf7ea-57d9-48d0-8353-b070f0719403  40.770344 -73.864433  MTABC_7451   
34454  ab5d1899-12b4-43fd-8c3c-edd3389bc38f  40.746563 -73.890571  MTABC_7449   
34452  fae77152-179e-42ee-8d03-dc3da799aeca  40.746563 -73.890571  MTABC_7449   
34450  0ae35cc2-32e2-45d1-bd19-1337b4022ce6  40.746563 -73.890571  MTABC_7449   
34448  d8b0a290-ccbd-4c4d-b80a-cf4b2a14d8af  40.746563 -73.890571  MTABC_7449   
34416  1b1451da-9ce9-4bc9-8da8-9c2e39fec131  40.774264 -73.871816  MTABC_7449   
34404  45ac0b68-4a30-46e9-9b1d-734b6c311cb3  40.745904 -73.903250  MTABC_7451   
34401  2783d683-c298-4f67-a09b-b27d44a41cd1  40.745904 -73.903250  MTABC_7451   
34402  add4f246-9458-4dc8-a75c-94abe4a6cd7a  40.770293 -73.864363  MTABC_7449   
34400  7efc454c-7ae8-48be-92d5-4ff26a667f27  40.745904 -73.903250  MTABC_7451   
34398  91f54904-a3e0-4af9-9fb8-c6178f023926  40.745904 -73.903250  MTABC_7451   
34396  561e94d7-0c7c-4217-8e2e-5654f66b5e4c  40.745904 -73.903250  MTABC_7451   
34393  4fb4248e-a060-4099-b88d-14e21a0e526d  40.745688 -73.902816  MTABC_7451   
34392  d846a86f-ae32-4502-8744-95fbbe3247cc  40.745688 -73.902816  MTABC_7451   
34390  6f71559e-6119-4caf-8ba6-0b820d3bb745  40.745688 -73.902816  MTABC_7451   
34388  68bc5467-f0d9-402b-af34-e54768dc4600  40.745688 -73.902816  MTABC_7451   
34378  9a8ca6a7-cdc2-4764-992f-a9f196be36c6  40.746682 -73.890449  MTABC_7451   
34366  7c265a84-d008-4409-8e6c-15a8631ef7d5  40.746894 -73.891271  MTABC_7449   
34364  bc0c6d8c-8d2f-45b8-ad54-a3f5e66d0b4d  40.746894 -73.891271  MTABC_7449   
34340  68e8d539-e52b-425e-a9ad-0b5fe35d24c8  40.774264 -73.871816  MTABC_7451   
34342  09e1e523-a887-41de-acb3-2cb1b1eac799  40.774264 -73.871816  MTABC_7451   
34338  c2f8caab-f144-4ecd-8c16-adf0a17a6108  40.745943 -73.903231  MTABC_7449   
...                                     ...        ...        ...         ...   
1210   e3f6f32b-6d78-4edd-817b-57a66cb18168  40.745904 -73.903250  MTABC_7448   
1593   e0db5465-bbe0-45cf-839a-f08a5b17a81c  40.745904 -73.903250  MTABC_7448   
1583   f120d75e-5f24-4538-b4d9-652912ebd62c  40.745904 -73.903250  MTABC_7448   
1573   1334b3f6-a59a-450b-99a7-a9a721f30e68  40.745904 -73.903250  MTABC_7448   
1563   0e406be6-2d09-47c4-b9b9-309b511b1c94  40.745904 -73.903250  MTABC_7448   
1553   01dd4ca5-fe3b-4e14-b641-d5e2449609c6  40.745904 -73.903250  MTABC_7448   
1544   70db651d-458c-48ac-9347-a1d45bdd2bca  40.745904 -73.903250  MTABC_7448   
1535   74f201f1-0bcb-4a6c-ae87-43aafbdcc0e6  40.745904 -73.903250  MTABC_7448   
1526   73e0e639-9c4a-41d0-bd61-f8eb6f267b4a  40.745904 -73.903250  MTABC_7448   
1516   14c80f22-3388-4c95-9049-099870f137b2  40.745904 -73.903250  MTABC_7448   
1201   37044a82-46f0-4573-b96c-1112ac9751bd  40.745904 -73.903250  MTABC_7448   
1191   a4cd5967-d39b-4689-bb0a-3cb5de593b6c  40.745904 -73.903250  MTABC_7448   
1181   085928e6-6129-4266-97eb-b0d69217d12f  40.745904 -73.903250  MTABC_7448   
1171   e9ad68a7-f46b-4ab7-84aa-f5fcfd85b647  40.745904 -73.903250  MTABC_7448   
1162   51c4d963-9922-4d01-8a6a-9c2ac755c350  40.745904 -73.903250  MTABC_7448   
1153   edf94bfb-21a3-4387-b7fe-84c1024f71fe  40.745904 -73.903250  MTABC_7448   
1071   298e4b31-5bce-46ea-903d-f514fd7f7e42  40.768370 -73.862554  MTABC_7455   
1054   c6067b6c-05c7-4361-bc1d-d639e5ab72df  40.770270 -73.864332  MTABC

# Calculate Duration by Time Interval Bins

- Filter to only include origin and destination stops. Calculate time delta between two time stamps

In [25]:
stops = ['MTA_553332', 'MTA_982075']
se = df1.loc[df1['next_stop_id'].isin(stops)]

In [26]:
se

id        lat        lon  vehicle_id  \
34612  f958a26b-b017-4654-ae01-c23851c65a33  40.745688 -73.902816  MTABC_7451   
34611  c9720755-0d32-48ed-9eb2-cfcb68d79ec3  40.745688 -73.902816  MTABC_7451   
34610  f851c396-afdb-4ef8-b37b-93f7bbde6af2  40.745688 -73.902816  MTABC_7451   
34609  5a2bdbe4-c4eb-4552-8230-bdfac12ba553  40.745688 -73.902816  MTABC_7451   
34608  eca20a41-4761-4b58-b6cc-ea7453f01631  40.745688 -73.902816  MTABC_7451   
34607  8b054d07-c9c8-4187-8570-00a02d5ea894  40.745688 -73.902816  MTABC_7451   
34404  45ac0b68-4a30-46e9-9b1d-734b6c311cb3  40.745904 -73.903250  MTABC_7451   
34401  2783d683-c298-4f67-a09b-b27d44a41cd1  40.745904 -73.903250  MTABC_7451   
34400  7efc454c-7ae8-48be-92d5-4ff26a667f27  40.745904 -73.903250  MTABC_7451   
34398  91f54904-a3e0-4af9-9fb8-c6178f023926  40.745904 -73.903250  MTABC_7451   
34396  561e94d7-0c7c-4217-8e2e-5654f66b5e4c  40.745904 -73.903250  MTABC_7451   
34393  4fb4248e-a060-4099-b88d-14e21a0e526d  40.745688 -73.902816  MTABC_7451   
34392  d846a86f-ae32-4502-8744-95fbbe3247cc  40.745688 -73.902816  MTABC_7451   
34390  6f71559e-6119-4caf-8ba6-0b820d3bb745  40.745688 -73.902816  MTABC_7451   
34388  68bc5467-f0d9-402b-af34-e54768dc4600  40.745688 -73.902816  MTABC_7451   
34338  c2f8caab-f144-4ecd-8c16-adf0a17a6108  40.745943 -73.903231  MTABC_7449   
34336  1d635842-b2f6-4ab9-b491-ae42ec45b91d  40.745943 -73.903231  MTABC_7449   
34334  b0364cab-222e-4622-a376-0ff645be6611  40.745943 -73.903231  MTABC_7449   
34332  01429dad-a00f-4b6a-943b-819ecd399ff6  40.745982 -73.903213  MTABC_7449   
34330  e4284988-2d33-4843-b72a-efbd87ad4751  40.745982 -73.903213  MTABC_7449   
34328  c276a86b-ad9e-44b6-b904-2fc64378c763  40.745982 -73.903213  MTABC_7449   
34325  0da0eb11-170e-4105-8f13-656755497a54  40.745982 -73.903213  MTABC_7449   
34324  7576738c-55ee-4553-89e5-817290620672  40.745982 -73.903213  MTABC_7449   
34322  e2e1816e-991c-4428-a93f-3b26b3708e56  40.745982 -73.903213  MTABC_7449   
34320  d1c6776a-dc74-476a-ab13-3774c5922867  40.745982 -73.903213  MTABC_7449   
34318  d11f4c8d-eeca-472a-bd2c-40e6579afb8f  40.745982 -73.903213  MTABC_7449   
34264  8eb4f918-836f-4854-b523-ae8627e0e83e  40.745904 -73.903250  MTABC_7451   
34262  ab9a3dff-5779-44f3-8b33-a9ce45c01463  40.745904 -73.903250  MTABC_7451   
34260  2af9b041-3b24-4cce-9b14-a55c79b741ca  40.745904 -73.903250  MTABC_7451   
34258  24d385ca-554a-440c-8af6-7258952d2696  40.745904 -73.903250  MTABC_7451   
...                                     ...        ...        ...         ...   
1306   7580583b-185a-43dc-a2e1-8c443e751e06  40.745904 -73.903250  MTABC_7448   
1257   73807eeb-f417-448d-8074-eef704138432  40.745904 -73.903250  MTABC_7448   
1238   44b9c1d2-50b1-4806-991e-8075c6b3ebc4  40.745904 -73.903250  MTABC_7448   
1227   f50c93a9-45ee-4884-9846-98a26f702298  40.745904 -73.903250  MTABC_7448   
1220   9dcaa5fe-6a1a-40cf-9ee5-068bcd6cfaf8  40.745904 -73.903250  MTABC_7448   
1210   e3f6f32b-6d78-4edd-817b-57a66cb18168  40.745904 -73.903250  MTABC_7448   
1593   e0db5465-bbe0-45cf-839a-f08a5b17a81c  40.745904 -73.903250  MTABC_7448   
1583   f120d75e-5f24-4538-b4d9-652912ebd62c  40.745904 -73.903250  MTABC_7448   
1573   1334b3f6-a59a-450b-99a7-a9a721f30e68  40.745904 -73.903250  MTABC_7448   
1563   0e406be6-2d09-47c4-b9b9-309b511b1c94  40.745904 -73.903250  MTABC_7448   
1553   01dd4ca5-fe3b-4e14-b641-d5e2449609c6  40.745904 -73.903250  MTABC_7448   
1544   70db651d-458c-48ac-9347-a1d45bdd2bca  40.745904 -73.903250  MTABC_7448   
1535   74f201f1-0bcb-4a6c-ae87-43aafbdcc0e6  40.745904 -73.903250  MTABC_7448   
1526   73e0e639-9c4a-41d0-bd61-f8eb6f267b4a  40.745904 -73.903250  MTABC_7448   
1516   14c80f22-3388-4c95-9049-099870f137b2  40.745904 -73.903250  MTABC_7448   
1201   37044a82-46f0-4573-b96c-1112ac9751bd  40.745904 -73.903250  MTABC_7448   
1191   a4cd5967-d39b-4689-bb0a-3cb5de593b6c  40.745904 -73.903250  MTABC_7448   
1181   085928e6-6129-4266-97eb-b0d69217d12f  40.745904 -73.903250  MTABC

In [27]:
dse = dict(tuple(se.groupby('vehicle_id')))

In [28]:
dse51 = dse['MTABC_7451']
dse51.reset_index(drop=True, inplace=True)
dse51['time_delta']=np.nan

dse49 = dse['MTABC_7449']
dse49.reset_index(drop=True, inplace=True)
dse49['time_delta']=np.nan

dse48 = dse['MTABC_7448']
dse48.reset_index(drop=True, inplace=True)
dse48['time_delta']=np.nan

dse50 = dse['MTABC_7450']
dse50.reset_index(drop=True, inplace=True)
dse50['time_delta']=np.nan

dse53 = dse['MTABC_7453']
dse53.reset_index(drop=True, inplace=True)
dse53['time_delta']=np.nan

dse46 = dse['MTABC_7446']
dse46.reset_index(drop=True, inplace=True)
dse46['time_delta']=np.nan

dse45 = dse['MTABC_7445']
dse45.reset_index(drop=True, inplace=True)
dse45['time_delta']=np.nan

dse72 = dse['MTABC_7472']
dse72.reset_index(drop=True, inplace=True)
dse72['time_delta']=np.nan

dse55 = dse['MTABC_7455']
dse55.reset_index(drop=True, inplace=True)
dse55['time_delta']=np.nan

dse63 = dse['MTABC_7463']
dse63.reset_index(drop=True, inplace=True)
dse63['time_delta']=np.nan


In [29]:
for i in range(0,(len(dse51) - 1)):
    dse51['time_delta'][i] = dse51.recorded_at_time[i+1] - dse51.recorded_at_time[i]
dse51

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
0    f958a26b-b017-4654-ae01-c23851c65a33  40.745688 -73.902816  MTABC_7451   
1    c9720755-0d32-48ed-9eb2-cfcb68d79ec3  40.745688 -73.902816  MTABC_7451   
2    f851c396-afdb-4ef8-b37b-93f7bbde6af2  40.745688 -73.902816  MTABC_7451   
3    5a2bdbe4-c4eb-4552-8230-bdfac12ba553  40.745688 -73.902816  MTABC_7451   
4    eca20a41-4761-4b58-b6cc-ea7453f01631  40.745688 -73.902816  MTABC_7451   
5    8b054d07-c9c8-4187-8570-00a02d5ea894  40.745688 -73.902816  MTABC_7451   
6    45ac0b68-4a30-46e9-9b1d-734b6c311cb3  40.745904 -73.903250  MTABC_7451   
7    2783d683-c298-4f67-a09b-b27d44a41cd1  40.745904 -73.903250  MTABC_7451   
8    7efc454c-7ae8-48be-92d5-4ff26a667f27  40.745904 -73.903250  MTABC_7451   
9    91f54904-a3e0-4af9-9fb8-c6178f023926  40.745904 -73.903250  MTABC_7451   
10   561e94d7-0c7c-4217-8e2e-5654f66b5e4c  40.745904 -73.903250  MTABC_7451   
11   4fb4248e-a060-4099-b88d-14e21a0e526d  40.745688 -73.902816  MTABC_7451   
12   d846a86f-ae32-4502-8744-95fbbe3247cc  40.745688 -73.902816  MTABC_7451   
13   6f71559e-6119-4caf-8ba6-0b820d3bb745  40.745688 -73.902816  MTABC_7451   
14   68bc5467-f0d9-402b-af34-e54768dc4600  40.745688 -73.902816  MTABC_7451   
15   8eb4f918-836f-4854-b523-ae8627e0e83e  40.745904 -73.903250  MTABC_7451   
16   ab9a3dff-5779-44f3-8b33-a9ce45c01463  40.745904 -73.903250  MTABC_7451   
17   2af9b041-3b24-4cce-9b14-a55c79b741ca  40.745904 -73.903250  MTABC_7451   
18   24d385ca-554a-440c-8af6-7258952d2696  40.745904 -73.903250  MTABC_7451   
19   0f5cde94-b6a7-4fb0-a5f6-749a79cde3a6  40.745904 -73.903250  MTABC_7451   
20   844d7ce7-c0f9-48a7-ac69-a06d54c00914  40.745904 -73.903250  MTABC_7451   
21   35a0a55d-c0e1-4c3d-88db-c6f90b302833  40.745904 -73.903250  MTABC_7451   
22   c99bb5d6-fa36-4145-8c07-5f1340aa574e  40.745904 -73.903250  MTABC_7451   
23   ab00e020-8946-4a57-b4e2-b6fbe7c01601  40.745904 -73.903250  MTABC_7451   
24   b04d1fb4-c9f5-484c-a7e3-dfc0f29c7951  40.745904 -73.903250  MTABC_7451   
25   362ac7b0-21d3-4dff-bf7a-07d1f73914bb  40.745904 -73.903250  MTABC_7451   
26   6de36455-7efd-481b-876c-bfc12dc8ce38  40.745786 -73.903307  MTABC_7451   
27   e6c88670-da7b-4796-a4dd-6f351393ae7f  40.745845 -73.903278  MTABC_7451   
28   38fbcd09-b4f8-4bf6-9503-a1c24cc96fc0  40.745894 -73.903255  MTABC_7451   
29   eab4932f-6a53-4993-8c54-6db37b0bd022  40.745894 -73.903255  MTABC_7451   
..                                    ...        ...        ...         ...   
262  9577fc98-6ade-4b79-951a-8655697a8802  40.745887 -73.903258  MTABC_7451   
263  0cd9b458-4d81-4142-8300-553eb64b59ef  40.745887 -73.903258  MTABC_7451   
264  6db5796b-73c2-4c50-8ca5-b502003cdea8  40.745887 -73.903258  MTABC_7451   
265  3b703691-bc0e-4a31-8375-8a5b74f698b0  40.745887 -73.903258  MTABC_7451   
266  73914769-76e5-46bd-adc7-6c09c099b25d  40.745887 -73.903258  MTABC_7451   
267  61b40fc0-6dd1-4e7a-a654-557633524214  40.745887 -73.903258  MTABC_7451   
268  69a4241a-43d6-468e-8a9d-2abeba05fb01  40.745887 -73.903258  MTABC_7451   
269  b13866ff-a1cb-467b-abc1-84e1eca86df3  40.745887 -73.903258  MTABC_7451   
270  3201a224-ba87-4cac-8029-1dc948dda5af  40.745887 -73.903258  MTABC_7451   
271  7f2ea977-801b-438e-9bbf-8eccc92c9acb  40.745887 -73.903258  MTABC_7451   
272  af0b0dcd-68e2-46d6-8d2f-7fbede0f3cb6  40.745887 -73.903258  MTABC_7451   
273  9d194936-af13-42ad-8f44-713b5fb266b7  40.745887 -73.903258  MTABC_7451   
274  adeb5686-1519-4fe7-b68f-7b5d5ea73607  40.745887 -73.903258  MTABC_7451   
275  d8e7383a-f5e1-4ac1-96b8-54e1802625b8  40.745887 -73.903258  MTABC_7451   
276  229a7e77-32c2-4a1b-9a04-b58db0337f5a  40.745887 -73.903258  MTABC_7451   
277  6fe9d354-48ff-4380-97df-b773d890ac6b  40.745887 -73.903258  MTABC_7451   
278  0c4c1ae3-f2b2-47c6-a010-62e7c25ce613  40.745887 -73.903258  MTABC_7451   
279  d1528ca6-b4fb-45f0-9401-dc54978d67b4  40.745887 -73.903258  MTABC_7451   
280  8e490b4a-8a45-429f-9dde-7e3999e96967  40.745887 -73.903258  MTABC_7451   
281  5ce3c

In [30]:
for i in range(0,(len(dse49) - 1)):
    dse49['time_delta'][i] = dse49.recorded_at_time[i+1] - dse49.recorded_at_time[i]
dse49

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
0    c2f8caab-f144-4ecd-8c16-adf0a17a6108  40.745943 -73.903231  MTABC_7449   
1    1d635842-b2f6-4ab9-b491-ae42ec45b91d  40.745943 -73.903231  MTABC_7449   
2    b0364cab-222e-4622-a376-0ff645be6611  40.745943 -73.903231  MTABC_7449   
3    01429dad-a00f-4b6a-943b-819ecd399ff6  40.745982 -73.903213  MTABC_7449   
4    e4284988-2d33-4843-b72a-efbd87ad4751  40.745982 -73.903213  MTABC_7449   
5    c276a86b-ad9e-44b6-b904-2fc64378c763  40.745982 -73.903213  MTABC_7449   
6    0da0eb11-170e-4105-8f13-656755497a54  40.745982 -73.903213  MTABC_7449   
7    7576738c-55ee-4553-89e5-817290620672  40.745982 -73.903213  MTABC_7449   
8    e2e1816e-991c-4428-a93f-3b26b3708e56  40.745982 -73.903213  MTABC_7449   
9    d1c6776a-dc74-476a-ab13-3774c5922867  40.745982 -73.903213  MTABC_7449   
10   d11f4c8d-eeca-472a-bd2c-40e6579afb8f  40.745982 -73.903213  MTABC_7449   
11   15e173fb-285d-440f-b7ed-f21f9419ef8b  40.745796 -73.903302  MTABC_7449   
12   1798d519-bc0e-42ee-85fd-1527d2fd4753  40.745796 -73.903302  MTABC_7449   
13   f47636b6-8848-40f0-ac5f-9b35d0a74c5c  40.745796 -73.903302  MTABC_7449   
14   05544940-7824-4e2c-85a9-acc7cf28252d  40.745796 -73.903302  MTABC_7449   
15   050be422-9547-49f8-ae94-44979e37d957  40.745746 -73.903326  MTABC_7449   
16   9e628c7d-5ef5-4f50-98fe-d48ea995fecb  40.745746 -73.903326  MTABC_7449   
17   0dd60deb-8fce-4ee2-aa24-f7850f08e27d  40.745746 -73.903326  MTABC_7449   
18   d0ac8fd0-d003-49d1-b8bb-728b64cfd69e  40.745746 -73.903326  MTABC_7449   
19   199d0fd4-d43d-4d5f-a735-bbac1e5825ae  40.745746 -73.903326  MTABC_7449   
20   54bd60a0-be25-4716-b252-15447598f202  40.745746 -73.903326  MTABC_7449   
21   5e4dbf79-4836-41b9-9fda-1c65f1ecb87c  40.745746 -73.903326  MTABC_7449   
22   96dd5fc9-ad36-4fcf-9e11-1f20c03f4384  40.745943 -73.903231  MTABC_7449   
23   5022b175-eeaa-40ab-b70c-e4c6af8a1d68  40.745992 -73.903208  MTABC_7449   
24   50a4c221-76e2-461a-8fd9-bfda0d61c7e9  40.745992 -73.903208  MTABC_7449   
25   5da59369-d61d-47c7-9f78-558b7da3fafb  40.745992 -73.903208  MTABC_7449   
26   7225d47d-eeab-4ab8-8ec7-bf8f1aafe8e5  40.745992 -73.903208  MTABC_7449   
27   d9694c81-93f4-484f-a24f-10d5dd318ca0  40.745992 -73.903208  MTABC_7449   
28   878e4763-a574-4216-8692-ee62a36f125a  40.745992 -73.903208  MTABC_7449   
29   05beddd1-b80f-4764-98d0-955f116aeb66  40.745992 -73.903208  MTABC_7449   
..                                    ...        ...        ...         ...   
256  db65b451-b74e-4f75-85ac-cec5dd5857f4  40.745803 -73.903299  MTABC_7449   
257  f2917bb8-7913-4768-b270-f17f8c4e4280  40.745803 -73.903299  MTABC_7449   
258  e43c71f4-0fa3-48a4-838e-c438ea219a3e  40.745803 -73.903299  MTABC_7449   
259  bb607600-6b81-4b2d-93b7-8a708b2b3e77  40.745803 -73.903299  MTABC_7449   
260  98384109-2b23-48b6-8b17-51322889119f  40.745803 -73.903299  MTABC_7449   
261  f6389a26-d8f7-437e-8e08-f94c8a5fa2cb  40.745803 -73.903299  MTABC_7449   
262  cb7aba72-2ff3-499b-ad00-3bd69b316e60  40.745803 -73.903299  MTABC_7449   
263  1aff8a7c-5afe-4e4f-8761-0df0d1353bd2  40.745803 -73.903299  MTABC_7449   
264  3789387a-d36c-40c2-a59a-cb98a9f24752  40.745803 -73.903299  MTABC_7449   
265  715a6dc3-1fff-41c4-a55f-72e586b440c7  40.745803 -73.903299  MTABC_7449   
266  a10ce043-1bf6-4002-b1dd-adcfb0236882  40.745803 -73.903299  MTABC_7449   
267  d9e78c96-b8b2-4f44-9e66-12104f72da27  40.745803 -73.903299  MTABC_7449   
268  e7eba126-535a-4997-b8a4-dccbb9a2d5b7  40.745803 -73.903299  MTABC_7449   
269  9ef302bf-a533-47f7-b985-cd38220a2d4d  40.745803 -73.903299  MTABC_7449   
270  9d717c00-a791-4566-a2ac-f1329c1b9bf2  40.745803 -73.903299  MTABC_7449   
271  5e7dd1bd-f4d6-446c-9536-68a2ff50ccbd  40.745803 -73.903299  MTABC_7449   
272  bdec3a2c-83ea-4a76-9b11-cc333a0ee0af  40.745803 -73.903299  MTABC_7449   
273  48e8e71c-2566-4c96-b87f-506598b661af  40.745803 -73.903299  MTABC_7449   
274  90fc458a-c607-40df-a5d3-cd5fd047308a  40.745803 -73.903299  MTABC_7449   
275  ab968

In [31]:
for i in range(0,(len(dse48) - 1)):
    dse48['time_delta'][i] = dse48.recorded_at_time[i+1] - dse48.recorded_at_time[i]
dse48

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
0    f63e602a-83a4-4b70-b14a-9388dd09eae8  40.745688 -73.902816  MTABC_7448   
1    c0c1190b-7679-438d-a625-f892332cfaea  40.745688 -73.902816  MTABC_7448   
2    2adf172b-0334-40b1-98da-44d0ecf4cc33  40.745688 -73.902816  MTABC_7448   
3    05b37cf9-e9be-4bed-bc84-fbc093af6799  40.745688 -73.902816  MTABC_7448   
4    aa88da1c-26e4-4334-acb4-d665f33b33b3  40.745688 -73.902816  MTABC_7448   
5    70382650-8f96-4911-a074-7f7525c9a143  40.745688 -73.902816  MTABC_7448   
6    e3b62c60-5914-4042-a094-f2451ff2b2b4  40.745943 -73.903231  MTABC_7448   
7    8c9297a1-a17a-4206-ae79-2da5c569c5f3  40.745796 -73.903302  MTABC_7448   
8    3ccaa1aa-8ef9-4a5e-a230-4e952eb98836  40.745688 -73.902816  MTABC_7448   
9    3f50251e-d32d-4094-823d-895f7ac11c77  40.745688 -73.902816  MTABC_7448   
10   c4efbd22-4da8-4a6b-bdbd-7ff635d6ef57  40.745688 -73.902816  MTABC_7448   
11   5ce200a5-17b4-4cd7-944a-87be160f9a58  40.745688 -73.902816  MTABC_7448   
12   010802f3-2c39-4c7f-ae5e-2d824085e853  40.745688 -73.902816  MTABC_7448   
13   1aab69f1-b7a4-41b5-90fc-3844648f25a3  40.745688 -73.902816  MTABC_7448   
14   270f78d4-10fb-45d2-8fa8-661349484603  40.745688 -73.902816  MTABC_7448   
15   1e330036-c8b3-4a50-a4e6-5f1c024c5699  40.745688 -73.902816  MTABC_7448   
16   7b759fa7-dc32-4185-bf73-432fb37d6cc2  40.745845 -73.903278  MTABC_7448   
17   45482fa4-890d-4555-bbb8-cdb6ef37bfae  40.745796 -73.903302  MTABC_7448   
18   d689e644-d5b9-4253-b905-1386f91ec19d  40.745796 -73.903302  MTABC_7448   
19   4bf38d66-5b10-455c-a150-088270c1413a  40.745796 -73.903302  MTABC_7448   
20   5c85c65f-79c4-4f2c-b9d1-83b8bc96429c  40.745796 -73.903302  MTABC_7448   
21   97d4fdfc-e505-465a-bb4a-4a82ec7a5b4c  40.745796 -73.903302  MTABC_7448   
22   3723e552-c1f0-4e5c-ba58-6371914bc3bb  40.745845 -73.903278  MTABC_7448   
23   0527676e-eb3f-4ea4-b1c8-d8c920910a18  40.745796 -73.903302  MTABC_7448   
24   f65255d5-c370-4754-8fdb-b76111e9b64c  40.745845 -73.903278  MTABC_7448   
25   60bd1805-b2f7-4d7e-ad4e-3a3e4085d0fe  40.745845 -73.903278  MTABC_7448   
26   78f40f7b-d7a6-49ba-a12f-15105927e9e1  40.745992 -73.903208  MTABC_7448   
27   16941734-f54d-4032-b184-2cf1c217b6e3  40.745845 -73.903278  MTABC_7448   
28   82e4fd04-ca7e-4dec-9bf7-c9941e27847c  40.745894 -73.903255  MTABC_7448   
29   df990fb8-125e-419e-93bc-bdff416d6c70  40.745904 -73.903250  MTABC_7448   
..                                    ...        ...        ...         ...   
161  e1b47020-6fbb-4065-83f6-293725a2d58b  40.745904 -73.903250  MTABC_7448   
162  ce6821d1-c28c-4db8-9fd7-93d58f9a04cd  40.745904 -73.903250  MTABC_7448   
163  e57e23c5-24b4-47c8-9f54-f1a60b5b3b7e  40.745904 -73.903250  MTABC_7448   
164  243dce93-da24-46ad-a8d8-b34875bd352b  40.745904 -73.903250  MTABC_7448   
165  02ec7c0c-4d4a-4884-826c-e89fcfebd555  40.745904 -73.903250  MTABC_7448   
166  404ef166-3897-465a-ace5-baaf3c51c38b  40.745904 -73.903250  MTABC_7448   
167  c08e6a0f-fd2b-46b6-ab43-d5ce192d3759  40.745904 -73.903250  MTABC_7448   
168  43dd93bb-b097-4e2d-9e19-e1e89fa69eef  40.745904 -73.903250  MTABC_7448   
169  a541b9d5-3378-47a8-aafd-9eb50f19a7f8  40.745904 -73.903250  MTABC_7448   
170  7580583b-185a-43dc-a2e1-8c443e751e06  40.745904 -73.903250  MTABC_7448   
171  73807eeb-f417-448d-8074-eef704138432  40.745904 -73.903250  MTABC_7448   
172  44b9c1d2-50b1-4806-991e-8075c6b3ebc4  40.745904 -73.903250  MTABC_7448   
173  f50c93a9-45ee-4884-9846-98a26f702298  40.745904 -73.903250  MTABC_7448   
174  9dcaa5fe-6a1a-40cf-9ee5-068bcd6cfaf8  40.745904 -73.903250  MTABC_7448   
175  e3f6f32b-6d78-4edd-817b-57a66cb18168  40.745904 -73.903250  MTABC_7448   
176  e0db5465-bbe0-45cf-839a-f08a5b17a81c  40.745904 -73.903250  MTABC_7448   
177  f120d75e-5f24-4538-b4d9-652912ebd62c  40.745904 -73.903250  MTABC_7448   
178  1334b3f6-a59a-450b-99a7-a9a721f30e68  40.745904 -73.903250  MTABC_7448   
179  0e406be6-2d09-47c4-b9b9-309b511b1c94  40.745904 -73.903250  MTABC_7448   
180  01dd4

In [32]:
for i in range(0,(len(dse50) - 1)):
    dse50['time_delta'][i] = dse50.recorded_at_time[i+1] - dse50.recorded_at_time[i]
dse50

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
0    eac91b59-ae40-426e-bd5f-55c85a11bb16  40.745894 -73.903255  MTABC_7450   
1    782ae290-f027-4ecc-9d20-194d3bef4a81  40.745845 -73.903278  MTABC_7450   
2    1b9f6b96-7ff3-4290-9c33-05e9ae3c9182  40.745845 -73.903278  MTABC_7450   
3    68a8b0d1-6aee-4318-ae7e-318936b304fc  40.745796 -73.903302  MTABC_7450   
4    de95115a-83df-4343-9f75-b43388942459  40.745845 -73.903278  MTABC_7450   
5    801218c1-47c9-4a1d-bb16-f63b5a2345bc  40.745894 -73.903255  MTABC_7450   
6    13a893d4-c439-44c2-9c26-2c8c7146a164  40.745894 -73.903255  MTABC_7450   
7    dd5f749a-b29f-484a-adb8-d3a8a569b8b3  40.745747 -73.903325  MTABC_7450   
8    76865ebb-7db5-49d1-b9cd-ed1c9264e6d1  40.745786 -73.903307  MTABC_7450   
9    d845c6fe-e5d7-4466-ab83-47855e9faa8a  40.745825 -73.903288  MTABC_7450   
10   6823ea6a-8cde-4393-957e-41a7962e6ec3  40.745786 -73.903307  MTABC_7450   
11   9dc056ac-5313-4106-a888-cf6240b75550  40.745786 -73.903307  MTABC_7450   
12   a52dd8a8-9881-4157-ba46-5ea9bdd8a17e  40.745825 -73.903288  MTABC_7450   
13   e6ed7447-5abb-4293-a210-c2cfbfecdafe  40.745825 -73.903288  MTABC_7450   
14   66a7fa1b-d0a3-49ba-b67d-5c8617f0d002  40.745786 -73.903307  MTABC_7450   
15   de74ebe5-03c3-45f1-bd71-2493367d33f1  40.745786 -73.903307  MTABC_7450   
16   458fc975-7272-435f-a02d-62bfa17e6bcc  40.745786 -73.903307  MTABC_7450   
17   ade8a65c-1510-450e-8891-92648458e6fc  40.745786 -73.903307  MTABC_7450   
18   4262c457-c14e-4c49-a839-1356498b484f  40.745786 -73.903307  MTABC_7450   
19   1dfe7626-cdb3-4908-9731-dbc73048ba17  40.745786 -73.903307  MTABC_7450   
20   ff3c8311-7c97-4b5b-939a-d59f0c8c0ba6  40.745786 -73.903307  MTABC_7450   
21   ccad00d5-533a-4fef-af00-491c60d5963d  40.745786 -73.903307  MTABC_7450   
22   0050e626-c631-415f-bb71-f7ff657e2f2d  40.745786 -73.903307  MTABC_7450   
23   3596b9fe-71c9-43bb-b216-9edc94aec154  40.745786 -73.903307  MTABC_7450   
24   367766e7-e207-453d-873d-a339782d5956  40.745786 -73.903307  MTABC_7450   
25   48fb97e5-6651-461d-9f16-e975cb3810ec  40.745746 -73.903326  MTABC_7450   
26   5119b7d6-62ce-4dc3-8dd2-601dd060c940  40.745845 -73.903278  MTABC_7450   
27   68bb74a1-8782-4ccf-89cd-9c65854c1843  40.745845 -73.903278  MTABC_7450   
28   3f47ae2c-2879-4024-9fdd-06968736e2ea  40.745845 -73.903278  MTABC_7450   
29   b8788f4c-dfe2-4587-9d1f-3b1e441d3ee9  40.745780 -73.903310  MTABC_7450   
..                                    ...        ...        ...         ...   
122  833c8568-1d88-4cb4-9ed8-fc6b12f2fdb1  40.745865 -73.903269  MTABC_7450   
123  0a7c400a-c929-432c-8256-7d912c46386b  40.745865 -73.903269  MTABC_7450   
124  b80d9087-509d-48bf-873f-3f6d4104206b  40.745865 -73.903269  MTABC_7450   
125  64a1df34-6547-43ed-9847-82d53517eca6  40.745865 -73.903269  MTABC_7450   
126  85e69c22-3853-41ac-aa33-7bf40aebb0d7  40.745865 -73.903269  MTABC_7450   
127  2e60dc55-5bd5-4f1b-b399-7525b52db736  40.745746 -73.903326  MTABC_7450   
128  54fa4752-c0c6-4641-a439-23142d8ac8f3  40.745982 -73.903213  MTABC_7450   
129  f0f29660-b248-4524-8c5e-50b332127fbd  40.745982 -73.903213  MTABC_7450   
130  49090536-80a0-48e0-a3f8-2d455de90c1f  40.745982 -73.903213  MTABC_7450   
131  53f8e783-55c4-4424-a1c6-a8a6f1ca2833  40.745982 -73.903213  MTABC_7450   
132  aa6e7669-fb88-4f83-aeb0-6b869bbdb13c  40.745982 -73.903213  MTABC_7450   
133  da0ec9be-4bda-4ac3-b951-2dc80624ba58  40.745982 -73.903213  MTABC_7450   
134  21ad95d7-f346-4bf5-abaa-30bc216471b8  40.745982 -73.903213  MTABC_7450   
135  148772cc-5747-4bc1-9032-0e65d1092a42  40.745982 -73.903213  MTABC_7450   
136  d48a409d-81af-4ba0-8103-8cedf455eb44  40.745982 -73.903213  MTABC_7450   
137  039d39cf-7722-421f-858a-332e23f20acf  40.745982 -73.903213  MTABC_7450   
138  f3c3d8c6-33f7-4c77-acbc-75ee3fdc213f  40.745982 -73.903213  MTABC_7450   
139  922de4ce-bc7c-4bc8-aa0f-9d18f8fefee7  40.745982 -73.903213  MTABC_7450   
140  eac1d703-139c-463d-a0df-fb6da0583f45  40.745982 -73.903213  MTABC_7450   
141  b8bbf

In [33]:
for i in range(0,(len(dse53) - 1)):
    dse53['time_delta'][i] = dse53.recorded_at_time[i+1] - dse53.recorded_at_time[i]
dse53

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
0    67c50d4e-7c2d-47b9-b320-5e33b2ea6019  40.745845 -73.903278  MTABC_7453   
1    6cb3d426-8c0e-4778-b7fc-eeddebab58f0  40.745688 -73.902816  MTABC_7453   
2    a4a6e2be-a4ac-4093-8210-0bf2664b5e6a  40.745825 -73.903288  MTABC_7453   
3    553a82c4-d419-4d14-a214-46ac329d80f9  40.745943 -73.903231  MTABC_7453   
4    9df389ee-a148-498f-b577-63acf3b3c386  40.745786 -73.903307  MTABC_7453   
5    fae4aba4-c6ef-42ed-9479-2ecc66bc48f9  40.745688 -73.902816  MTABC_7453   
6    61091081-e42f-4809-9555-55f093cfdb25  40.745688 -73.902816  MTABC_7453   
7    84b16396-2306-46ec-bbce-692ed10fa88a  40.745982 -73.903213  MTABC_7453   
8    91886d44-6a63-4ff4-b92c-99df1f93dbcc  40.745982 -73.903213  MTABC_7453   
9    4a0b6c7e-a1f0-425e-9f7d-532a7607abea  40.745982 -73.903213  MTABC_7453   
10   66ab3bac-8abc-4348-90c1-8e111b5afedc  40.745982 -73.903213  MTABC_7453   
11   6aec2cb4-4f9f-4bb5-8816-f0aaca9dae05  40.745982 -73.903213  MTABC_7453   
12   f8010b36-06a2-4885-aacd-fb58d9e8cf0a  40.745982 -73.903213  MTABC_7453   
13   d572bf7f-9b43-4310-9475-e533b90a304e  40.745982 -73.903213  MTABC_7453   
14   4fca4118-ca8f-44be-8d0a-8a03592bd29b  40.745982 -73.903213  MTABC_7453   
15   ba2b69c2-a44d-48d1-b685-4673b7e1024a  40.745982 -73.903213  MTABC_7453   
16   068a2381-bda6-4f0d-8e57-9d83eacaa411  40.745786 -73.903307  MTABC_7453   
17   3b77af5e-4a45-41eb-9558-374546ca1d1c  40.745688 -73.902816  MTABC_7453   
18   7f488165-5e21-478d-a42a-910d546076ea  40.745976 -73.903216  MTABC_7453   
19   dded0f77-f6dd-4c5c-b510-9c5dcb0cd930  40.745943 -73.903231  MTABC_7453   
20   f762d660-130c-482a-8779-7e686cdf3390  40.745943 -73.903231  MTABC_7453   
21   7212b3f1-4fc2-42e3-8296-c03d53d3d8b2  40.745943 -73.903231  MTABC_7453   
22   f3330697-52e8-4ac9-936c-85f7a44f2f58  40.745943 -73.903231  MTABC_7453   
23   138aafcb-767a-409c-be47-b6330bfb8883  40.745943 -73.903231  MTABC_7453   
24   daa153f8-ce84-45e7-a0dd-055c7849b545  40.745943 -73.903231  MTABC_7453   
25   55c8e860-133f-4387-98f8-e110e1df8e51  40.745943 -73.903231  MTABC_7453   
26   b2c68555-fd19-4d84-9527-bb0d46fb5e11  40.745943 -73.903231  MTABC_7453   
27   d971ee0d-8d92-4b1e-adc9-57eee4d984d7  40.745943 -73.903231  MTABC_7453   
28   7e103946-fd97-4366-bf34-22c3d00dc931  40.745845 -73.903278  MTABC_7453   
29   34e12b15-3107-4482-8c31-7bb77dd46512  40.745845 -73.903278  MTABC_7453   
..                                    ...        ...        ...         ...   
111  8dc53aa9-b530-44b4-b9d4-d37873acd0e6  40.745775 -73.903312  MTABC_7453   
112  9d1db50b-cfc8-46ba-9a42-6b7fa6e68917  40.745775 -73.903312  MTABC_7453   
113  2daac4e0-f097-4da3-99b8-f4720bb52ec9  40.745775 -73.903312  MTABC_7453   
114  3c86d006-e542-479d-91c2-c464f1bb8598  40.745775 -73.903312  MTABC_7453   
115  df2cec2f-e249-4c45-b9f8-7ff6268a57ee  40.745775 -73.903312  MTABC_7453   
116  478f0f70-e262-45d9-b67d-45d62fd70f57  40.745775 -73.903312  MTABC_7453   
117  cb3975b0-534b-404d-8035-0a0410275c2d  40.745746 -73.903326  MTABC_7453   
118  90b138bd-a911-4951-9ddc-8b1bb7d0048a  40.745746 -73.903326  MTABC_7453   
119  b26547a9-99f0-41f8-a7db-0c09468717c1  40.745688 -73.902816  MTABC_7453   
120  b85f1993-f11c-4f80-a828-314e278dd5eb  40.745943 -73.903231  MTABC_7453   
121  5dfce0a6-043d-4701-9810-6d54134457eb  40.745688 -73.902816  MTABC_7453   
122  043736b1-0d81-494b-b8f7-9546d89dcb94  40.745943 -73.903231  MTABC_7453   
123  b88e17fa-4ced-4e33-be47-ad24467278f2  40.745982 -73.903213  MTABC_7453   
124  bd130d22-94aa-47e9-a188-5ac83724a68d  40.745982 -73.903213  MTABC_7453   
125  81e88b67-e5fa-4f1e-a15b-80f183aac0e4  40.745982 -73.903213  MTABC_7453   
126  16c3e33a-5585-4018-b341-780a940c4942  40.745982 -73.903213  MTABC_7453   
127  d3f9421e-bfd4-433f-8e49-85364c44c6e5  40.745982 -73.903213  MTABC_7453   
128  9c3dde87-b808-4770-abc2-445ecab42e8b  40.745982 -73.903213  MTABC_7453   
129  6d08e007-8264-4a5f-9653-5ccda24c911f  40.745982 -73.903213  MTABC_7453   
130  53fad

In [34]:
for i in range(0,(len(dse46) - 1)):
    dse46['time_delta'][i] = dse46.recorded_at_time[i+1] - dse46.recorded_at_time[i]
dse46

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
0    ec71025d-74ef-4c76-941e-162790f226ce  40.745786 -73.903307  MTABC_7446   
1    49e378ba-279b-48df-be8b-0ea6271a8b93  40.745865 -73.903269  MTABC_7446   
2    b6b61f05-56c6-4e01-9f65-690ab45b47a1  40.745688 -73.902816  MTABC_7446   
3    eac0eb59-f4fe-4cf5-a033-a377092c8ea0  40.745688 -73.902816  MTABC_7446   
4    2041f32b-6582-4c34-b8d7-b180fc147c91  40.745688 -73.902816  MTABC_7446   
5    05c49433-a7a2-48e0-8c04-34351744d0a5  40.745688 -73.902816  MTABC_7446   
6    76b80126-ced2-4d96-b501-e302fe263b5f  40.745688 -73.902816  MTABC_7446   
7    b81b763c-f8a6-4016-9743-0eab2be5f558  40.745688 -73.902816  MTABC_7446   
8    86e09af4-c926-42ff-8f51-53ce5ed8c4c1  40.745688 -73.902816  MTABC_7446   
9    50f54b28-bf3b-4135-b136-0c5f3a0f825c  40.745688 -73.902816  MTABC_7446   
10   582131e4-2a40-4c9e-8237-33a7fee85cc7  40.745688 -73.902816  MTABC_7446   
11   c6f17450-96b5-4f65-baa7-e2349f630453  40.745688 -73.902816  MTABC_7446   
12   0471bfcf-c2fb-4bda-ba9d-53f36d6492d7  40.745688 -73.902816  MTABC_7446   
13   033dd8db-141c-4ab2-82da-87191680c697  40.745688 -73.902816  MTABC_7446   
14   149b2b71-d283-490b-a46a-4342da0c0b11  40.745688 -73.902816  MTABC_7446   
15   14e002e2-8d41-41b5-8f28-4723c972b021  40.745688 -73.902816  MTABC_7446   
16   311a9227-1eaa-4b08-ac8e-b52133f2ba95  40.745943 -73.903231  MTABC_7446   
17   0fd24240-7fab-4aeb-8b6d-a1d3b4b9483d  40.745688 -73.902816  MTABC_7446   
18   97ac9409-64f3-4489-bc48-4b59650aa690  40.745976 -73.903216  MTABC_7446   
19   c76aebe7-9de2-45e3-b25c-0404a4e2bb9d  40.745910 -73.903247  MTABC_7446   
20   ac55b2b4-337d-4907-8dcb-aadc84cc225a  40.745910 -73.903247  MTABC_7446   
21   5e32570a-47fb-4787-b16e-24de3e078a72  40.745780 -73.903310  MTABC_7446   
22   25129caf-5bf9-468a-8b63-f3c050839057  40.745976 -73.903216  MTABC_7446   
23   9a3afc97-3dda-4daa-8d50-6b2113ee2473  40.745976 -73.903216  MTABC_7446   
24   1dea9a01-be7e-4b46-859a-93357acf74da  40.745976 -73.903216  MTABC_7446   
25   7117e369-8c90-4024-80d5-aa2fd0560a53  40.745976 -73.903216  MTABC_7446   
26   d2eb650f-799e-4a17-bdcf-05bc5541da21  40.745976 -73.903216  MTABC_7446   
27   ef014a9f-2f80-4ef3-bf79-ac008471d349  40.745976 -73.903216  MTABC_7446   
28   138bf606-b5e4-448e-9529-d7e39450b50b  40.745976 -73.903216  MTABC_7446   
29   6e9104fa-de2e-4575-b3b1-193208d798b7  40.745976 -73.903216  MTABC_7446   
..                                    ...        ...        ...         ...   
180  443de024-f3ea-44c1-9bdf-98d66612c4e6  40.745803 -73.903299  MTABC_7446   
181  cd477ccd-393f-41c0-b1f7-f503c11f4efb  40.745803 -73.903299  MTABC_7446   
182  3c813ce9-97c2-4a69-95b6-d4f2fa5e7a91  40.745803 -73.903299  MTABC_7446   
183  80425277-b708-459f-b174-b347f2f7da43  40.745803 -73.903299  MTABC_7446   
184  93fa2c79-3bb9-4af3-b2b7-d7c5656cfbf6  40.745803 -73.903299  MTABC_7446   
185  9ee3623b-8085-4440-ac96-7a24f92bc238  40.745803 -73.903299  MTABC_7446   
186  3feebd09-ab52-42f1-bc58-21edb9c47bbe  40.745803 -73.903299  MTABC_7446   
187  5d00d88c-2e89-4c19-b746-685c3c0337b9  40.745803 -73.903299  MTABC_7446   
188  c2350d3e-1adc-4700-afc6-c8a50471911a  40.745803 -73.903299  MTABC_7446   
189  8d83cd7b-a6d5-45f0-b130-7bddada1a92d  40.745803 -73.903299  MTABC_7446   
190  524ca662-927d-4a74-bf42-d30c9f9cca3b  40.745803 -73.903299  MTABC_7446   
191  8f7bc412-f917-4da2-a665-264ecd0aee81  40.745803 -73.903299  MTABC_7446   
192  d865018d-3d8d-4472-a595-1f8699749337  40.745803 -73.903299  MTABC_7446   
193  d722cf5e-2ddf-451a-b622-ba6cb4783ff2  40.745803 -73.903299  MTABC_7446   
194  98f5ed37-3b56-42b4-8b4e-43cd925b73b9  40.745803 -73.903299  MTABC_7446   
195  887e35f0-a9f8-45cc-bc9a-c45ca800dd96  40.745803 -73.903299  MTABC_7446   
196  1c1a69cf-8af5-4f46-aa90-0f7786516093  40.745803 -73.903299  MTABC_7446   
197  cff1ea16-ca00-4604-8af8-c2390c12cbf7  40.745803 -73.903299  MTABC_7446   
198  3fdfa97e-04ec-43b7-8fac-ab9fc42dd387  40.745803 -73.903299  MTABC_7446   
199  2a052

In [35]:
for i in range(0,(len(dse45) - 1)):
    dse45['time_delta'][i] = dse45.recorded_at_time[i+1] - dse45.recorded_at_time[i]
dse45

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
0    a29849c7-aa64-400c-b40b-ed12ddc5d27e  40.745943 -73.903231  MTABC_7445   
1    a5ea0459-0a94-47b8-8570-0e40e812c12c  40.745865 -73.903269  MTABC_7445   
2    7ca00c24-35c1-4f21-869c-b3fbc929a79f  40.745865 -73.903269  MTABC_7445   
3    38c80731-7777-4c32-8bb7-7ab9fb188f34  40.745865 -73.903269  MTABC_7445   
4    a10c4667-7ccd-4037-a07a-0791223f50ca  40.745865 -73.903269  MTABC_7445   
5    01948521-be52-4986-a2dd-c18b29a26390  40.745865 -73.903269  MTABC_7445   
6    01b2a7e3-6bd0-485b-804a-367c3570f1b8  40.745865 -73.903269  MTABC_7445   
7    5115b1c5-6df0-4dea-b553-ab2c05e791d3  40.745865 -73.903269  MTABC_7445   
8    895e7bf8-bacd-4a10-aacf-2618efab9e57  40.745865 -73.903269  MTABC_7445   
9    c89d45b4-00a3-4e85-888c-11a328efb878  40.745865 -73.903269  MTABC_7445   
10   074a7c26-13c2-4b04-b5ef-781df85c82bf  40.745865 -73.903269  MTABC_7445   
11   e7a047ab-1abd-48f8-89c2-d8ee9c0eca08  40.745746 -73.903326  MTABC_7445   
12   bcd349dd-0298-4e82-94ba-e3786de4e2a6  40.745865 -73.903269  MTABC_7445   
13   cc4fed7d-6ff1-4572-8497-8563bd06bbcf  40.745865 -73.903269  MTABC_7445   
14   335df4fd-8218-411f-8e9c-e028030f9cc0  40.745865 -73.903269  MTABC_7445   
15   5fd0076c-71d6-4538-bd53-ab39209c6bd2  40.745865 -73.903269  MTABC_7445   
16   63433d9f-a1a3-46b0-98e2-6546e45e63bc  40.745865 -73.903269  MTABC_7445   
17   97cd1556-bb16-4159-8f89-ad7c3437f582  40.745865 -73.903269  MTABC_7445   
18   f568444f-1831-4b16-9bf3-e2725d81b480  40.745865 -73.903269  MTABC_7445   
19   2e48cc48-89e5-4210-bebd-32996e28d629  40.745865 -73.903269  MTABC_7445   
20   7819da93-a2ea-4d07-ba41-d3479fafb30e  40.745865 -73.903269  MTABC_7445   
21   04a95986-373c-430a-8fcb-51d042ab7b30  40.745865 -73.903269  MTABC_7445   
22   512d7301-3d2f-4831-bccc-77db55d277ff  40.745865 -73.903269  MTABC_7445   
23   be17a3a0-bc44-4e7b-92dc-ac95115f2e17  40.745688 -73.902816  MTABC_7445   
24   e0b1c5a5-0d9b-4161-9a8c-7eb81cff52cc  40.745865 -73.903269  MTABC_7445   
25   1ede74e5-0691-475b-81fd-540fda193b5f  40.745865 -73.903269  MTABC_7445   
26   79ed8068-e21a-4d8c-a0f3-4acc690f7f9f  40.745865 -73.903269  MTABC_7445   
27   cf251f6a-2e23-49a1-8794-226fbe910a2d  40.745865 -73.903269  MTABC_7445   
28   2f6f9d8f-d250-436f-adea-f464b43e5c4a  40.745865 -73.903269  MTABC_7445   
29   14c27431-a187-43ee-b1c6-ca3f31c603f6  40.745865 -73.903269  MTABC_7445   
..                                    ...        ...        ...         ...   
112  6300c2c5-7692-404b-8470-44918fb58321  40.745845 -73.903278  MTABC_7445   
113  11f5fc11-172c-4b51-818d-29506718dd9b  40.745845 -73.903278  MTABC_7445   
114  63f2e4fc-f16e-42fc-901c-5347e7adf857  40.745746 -73.903326  MTABC_7445   
115  69a1c0b8-465d-4517-8133-d7eda95a879f  40.745746 -73.903326  MTABC_7445   
116  cbc9bd09-8a47-4b0f-9f99-9f9d6ebed26f  40.745746 -73.903326  MTABC_7445   
117  a70294d3-d2aa-454d-b1b7-b4059b68926e  40.745746 -73.903326  MTABC_7445   
118  6790c8bb-e084-486c-b45e-4f707e023397  40.745746 -73.903326  MTABC_7445   
119  53f5bfac-1115-4a6c-8cac-7c01828df8be  40.745746 -73.903326  MTABC_7445   
120  fa6b2cce-d839-4b34-9b5a-63e9d3277ce9  40.745971 -73.903218  MTABC_7445   
121  695c19ad-fe59-4821-abf0-e80e032d27ca  40.745887 -73.903258  MTABC_7445   
122  d3537b89-6cc1-4638-a2ae-58e821110f04  40.745803 -73.903299  MTABC_7445   
123  fe5983ce-477b-4f52-8fff-b85db15eff31  40.745803 -73.903299  MTABC_7445   
124  0f0f8423-280c-4719-9f47-2be73281e435  40.745803 -73.903299  MTABC_7445   
125  ae79fa1e-8a94-41b3-a27d-612256d496d8  40.745803 -73.903299  MTABC_7445   
126  7f0d796c-597c-4378-89f6-27c9728503b3  40.745688 -73.902816  MTABC_7445   
127  22f2e0e9-ab09-40b7-bbe9-9a3c3260b3a9  40.745915 -73.903245  MTABC_7445   
128  f0b8aab9-bc9e-41f4-95f3-8d10e77af72f  40.745775 -73.903312  MTABC_7445   
129  2923ae4a-4217-405a-8659-1dee2ded1903  40.745775 -73.903312  MTABC_7445   
130  fb9e7973-286c-4be5-970e-7b31884c4e06  40.745775 -73.903312  MTABC_7445   
131  01e57

In [36]:
for i in range(0,(len(dse72) - 1)):
    dse72['time_delta'][i] = dse72.recorded_at_time[i+1] - dse72.recorded_at_time[i]
dse72

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
0   9577f421-23c2-4339-ade2-59cb098e98b2  40.745999 -73.903205  MTABC_7472   
1   18bd6a17-f957-4392-b305-c178d8be8734  40.745915 -73.903245  MTABC_7472   
2   fcdf6d01-7d30-4c34-a61c-08041ec72ef2  40.745775 -73.903312  MTABC_7472   
3   2c23cb75-a2dd-4874-a0bc-8624f6517969  40.745803 -73.903299  MTABC_7472   
4   467e8a29-ea69-425b-93a6-ddf9b034b79f  40.745803 -73.903299  MTABC_7472   
5   74f6cfad-edbd-4bb9-8cbf-22ca837f112d  40.745803 -73.903299  MTABC_7472   
6   45773d84-486f-4e35-8e26-cdd64564718e  40.745803 -73.903299  MTABC_7472   
7   93bc915e-73f8-4696-a970-1b6bfb032c70  40.745803 -73.903299  MTABC_7472   
8   82a8d811-659b-4446-b860-e05b2281119e  40.745803 -73.903299  MTABC_7472   
9   90338693-9dff-41eb-802a-399857fed727  40.745859 -73.903272  MTABC_7472   
10  f9f9d25a-5f61-4fa3-8518-72a701f0299a  40.745831 -73.903285  MTABC_7472   
11  dbacc65b-f38d-44fa-932c-79449f77b07e  40.745831 -73.903285  MTABC_7472   
12  56a6d6ba-5534-4335-803a-c0e92386d9c4  40.745746 -73.903326  MTABC_7472   
13  c5adf530-e639-40ac-be6c-163730ef7973  40.745915 -73.903245  MTABC_7472   
14  f8c9e1f8-1a2e-468b-9052-7745d540d61d  40.745915 -73.903245  MTABC_7472   
15  8647bbe6-917c-4a75-8fba-4e2ebbbc0b8c  40.745915 -73.903245  MTABC_7472   
16  665b6f17-878f-43ce-ae8a-5edc24fcdeab  40.745915 -73.903245  MTABC_7472   
17  ee41fcca-d80e-4bd1-a4d1-190f6859634f  40.745915 -73.903245  MTABC_7472   
18  01a39b8b-2646-4d4b-8263-7e241e9eddc3  40.745915 -73.903245  MTABC_7472   
19  5ac0746a-15a3-4b45-a718-c657807e1790  40.745915 -73.903245  MTABC_7472   
20  b7b1106a-f1fc-4919-954f-841ee9ebbbba  40.745915 -73.903245  MTABC_7472   
21  c538bdfd-1c53-4083-b3ca-5074e1c81f8f  40.745915 -73.903245  MTABC_7472   
22  9592b3fa-b696-4618-844b-beb15500c7b2  40.745915 -73.903245  MTABC_7472   
23  77161a4b-c65e-4cd9-a4e1-efb94c3f163d  40.745915 -73.903245  MTABC_7472   
24  07e1a506-822f-48eb-a4dc-4c41e0ca7eb2  40.745915 -73.903245  MTABC_7472   
25  c65d308e-bb7f-4fb1-a586-92870ca8098f  40.745915 -73.903245  MTABC_7472   
26  3320bf9c-8bf0-42b1-9465-42771184f556  40.745803 -73.903299  MTABC_7472   
27  14cf5316-ef99-4b15-8bd6-3deba1ad06c4  40.745746 -73.903326  MTABC_7472   

   next_stop_id next_predicted_arrival_time next_predicted_departure_time  \
0    MTA_982075     2017-12-10 15:37:12.302    2017-12-10 15:37:12.302000   
1    MTA_982075     2017-12-10 15:37:42.352    2017-12-10 15:37:42.352000   
2    MTA_982075     2017-12-10 15:38:12.288    2017-12-10 15:38:12.288000   
3    MTA_982075                         NaT                           NaN   
4    MTA_982075     2017-12-10 15:39:42.423    2017-12-10 15:39:42.423000   
5    MTA_982075     2017-12-10 15:38:42.349    2017-12-10 15:38:42.349000   
6    MTA_982075                         NaT                           NaN   
7    MTA_982075     2017-12-10 15:39:12.402    2017-12-10 15:39:12.402000   
8    MTA_982075     2017-12-10 15:40:12.420    2017-12-10 15:40:12.420000   
9    MTA_982075     2017-12-10 16:56:44.576    2017-12-10 16:56:44.576000   
10   MTA_982075     2017-12-10 16:57:14.524    2017-12-10 16:57:14.524000   
11   MTA_982075     2017-12-10 16:57:44.564    2017-12-10 16:57:44.564000   
12   MTA_553332                         NaT                           NaN   
13   MTA_982075                         NaT                           NaN   
14   MTA_982075     2017-12-10 20:09:50.076    2017-12-10 20:09:50.076000   
15   MTA_982075     2017-12-10 20:10:19.884    2017-12-10 20:10:19.884000   
16   MTA_982075     2017-12-10 20:10:50.067    2017-12-10 20:10:50.067000   
17   MTA_982075     2017-12-10 20:11:19.965    2017-12-10 20:11:19.965000   
18   MTA_982075     2017-12-10 20:11:49.955    2017-12-10 20:11:49.955000   
19   MTA_982075     2017-12-10 20:12:19.957    2017-12-10 20:12:19.957000   
20   MTA_982075     2017-12-10 20:12:49.954    2017-12-10 20:12:49.954000   
21   MTA_982075     2017-12-10 20:13:19.988    2017-12-10 20:13:19.988000   
22  

In [37]:
for i in range(0,(len(dse55) - 1)):
    dse55['time_delta'][i] = dse55.recorded_at_time[i+1] - dse55.recorded_at_time[i]
dse55

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
0    f2dbad95-8ca8-4ec2-b793-0396c4c0b6d5  40.745999 -73.903205  MTABC_7455   
1    be4d441b-ffb8-4fb1-b53b-835b96e0afcd  40.745943 -73.903231  MTABC_7455   
2    eed395bf-e555-4b7c-8b1c-c2c0f7be67aa  40.745943 -73.903231  MTABC_7455   
3    8df24e82-f9ff-40ef-99db-1c7f3fa4a7fc  40.745943 -73.903231  MTABC_7455   
4    7142d25f-2618-4923-9a9b-fc62577e86de  40.745943 -73.903231  MTABC_7455   
5    40820fde-7fb3-4e56-a337-b91d4fcab1fd  40.745943 -73.903231  MTABC_7455   
6    16a1ad33-a3ae-4b2c-9e16-1d96a77fdf99  40.745943 -73.903231  MTABC_7455   
7    ce0ae10b-a2f2-42e9-8017-cd68d6e35023  40.745943 -73.903231  MTABC_7455   
8    0f24de10-a851-4b47-b096-09292a3b358a  40.745943 -73.903231  MTABC_7455   
9    194adff0-3721-42e2-94a6-3d0f891e78fe  40.745943 -73.903231  MTABC_7455   
10   61243742-4e02-4d25-bd68-24fecf38ccf5  40.745943 -73.903231  MTABC_7455   
11   7a8e74e1-0d4b-47f6-b321-cf3ebdd8edfc  40.745943 -73.903231  MTABC_7455   
12   fe10d153-91b3-4832-b3bd-cd2696d1c930  40.745943 -73.903231  MTABC_7455   
13   1afc04f1-3d43-413c-a635-49fcce92ac11  40.745943 -73.903231  MTABC_7455   
14   96e1e41d-53fc-4313-891b-27e19f82b830  40.745943 -73.903231  MTABC_7455   
15   2de369cf-71a2-4fc8-b2e0-2a69c25c9830  40.745915 -73.903245  MTABC_7455   
16   0a2a1821-4652-498d-bd66-00bc117da362  40.745915 -73.903245  MTABC_7455   
17   cd57e1dc-32c6-4608-b360-3153900b07b2  40.745915 -73.903245  MTABC_7455   
18   34d50407-3f0c-4b25-b0dd-d7dd1c5f018a  40.745915 -73.903245  MTABC_7455   
19   b2ccfc40-764e-49c5-aea5-03d9fb18408d  40.745775 -73.903312  MTABC_7455   
20   d0980607-d26f-424b-bada-fa1cef1b1905  40.745775 -73.903312  MTABC_7455   
21   34794b33-2180-41a3-92d1-b78dbc068674  40.745775 -73.903312  MTABC_7455   
22   6019061b-951b-4e9a-a8b2-47fb9820b267  40.745746 -73.903326  MTABC_7455   
23   cc83d2e1-2731-4531-9c03-0b74ba214ea5  40.745943 -73.903231  MTABC_7455   
24   8c5a37e4-2b57-49bb-a67e-d3642f4fb9d7  40.745943 -73.903231  MTABC_7455   
25   31ba7616-1909-4b19-b761-348e82d4eeff  40.745943 -73.903231  MTABC_7455   
26   f224cf85-e2ae-4bbb-977e-380437f11511  40.745943 -73.903231  MTABC_7455   
27   a686f88b-7915-4834-ada1-445850f6bd3b  40.745775 -73.903312  MTABC_7455   
28   ea0bbaf2-4d1a-4d54-afa0-fc3813cf6c27  40.745999 -73.903205  MTABC_7455   
29   83445e2e-214f-4eaa-865e-45dea2fca3f1  40.745971 -73.903218  MTABC_7455   
..                                    ...        ...        ...         ...   
114  4e768ae8-977e-4a2e-bd91-5405d68001bb  40.745904 -73.903250  MTABC_7455   
115  a558d373-7cd2-4c78-8901-6997155a3d83  40.745904 -73.903250  MTABC_7455   
116  510475c0-db9e-4b29-95a5-63f121d4826d  40.745825 -73.903288  MTABC_7455   
117  f3c40f34-eb30-4baf-9cc1-859dd9832e32  40.745825 -73.903288  MTABC_7455   
118  57ed661f-7876-4798-8736-23120ee07cb4  40.745825 -73.903288  MTABC_7455   
119  5f0cc6d3-526a-4d05-a735-d72cbcba751a  40.745786 -73.903307  MTABC_7455   
120  d8571a71-015d-4cbc-a066-65c47d209e17  40.745786 -73.903307  MTABC_7455   
121  657932f1-7498-4dda-b99b-06f58d2f0b6c  40.745786 -73.903307  MTABC_7455   
122  57c95b54-8caa-4340-a1dd-a66c153c9bc4  40.745786 -73.903307  MTABC_7455   
123  aa530830-b56e-4d01-a20a-5b10a0cd49eb  40.745786 -73.903307  MTABC_7455   
124  4c536ed5-7ed9-4d74-b41c-7a72b9cc04eb  40.745786 -73.903307  MTABC_7455   
125  754f8ebc-0c4b-44fb-bc99-7940094ea678  40.745786 -73.903307  MTABC_7455   
126  547e054d-d471-42d7-b69f-411bf874e57f  40.745786 -73.903307  MTABC_7455   
127  185814e6-21e7-45fe-8dea-c4f77dbed7f7  40.745786 -73.903307  MTABC_7455   
128  7f8b932d-bf93-468b-80f6-f8160d93c8b5  40.745746 -73.903326  MTABC_7455   
129  edbc8b9c-c477-411e-8118-bd485ab45908  40.745982 -73.903213  MTABC_7455   
130  ad3b340e-251e-48df-886d-76ca1061a184  40.745982 -73.903213  MTABC_7455   
131  02825e23-7869-40fe-ada6-75d94356a68d  40.745982 -73.903213  MTABC_7455   
132  04c4970d-22e5-4652-a8a9-813cc9165dad  40.745786 -73.903307  MTABC_7455   
133  d22f2

In [38]:
for i in range(0,(len(dse63) - 1)):
    dse63['time_delta'][i] = dse63.recorded_at_time[i+1] - dse63.recorded_at_time[i]
dse63

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


id        lat        lon  vehicle_id  \
0   05f7ee13-48d1-48fe-8628-37f624d1d2b6  40.745915 -73.903245  MTABC_7463   
1   a013098e-267b-42c4-916e-cab39b2f0563  40.745775 -73.903312  MTABC_7463   
2   c05379bb-9dfc-45a0-aa26-a747ea73b654  40.745688 -73.902816  MTABC_7463   
3   0b4579c2-d59f-4aae-bd58-30374086df4f  40.745865 -73.903269  MTABC_7463   
4   c4d5b098-7bee-441a-b018-ade316258264  40.745786 -73.903307  MTABC_7463   
5   e37dfbf6-d35b-4205-841c-081e3aaad313  40.745786 -73.903307  MTABC_7463   
6   57fe32f8-ea3e-46a1-aa0f-8c31745b1550  40.745786 -73.903307  MTABC_7463   
7   2b6858da-72c7-465e-8876-ef21a74ebca7  40.745786 -73.903307  MTABC_7463   
8   c3198771-960c-4c60-8ac8-dd8bcf76a76a  40.745746 -73.903326  MTABC_7463   
9   5c953290-7601-41c7-9876-496388af08ad  40.745746 -73.903326  MTABC_7463   
10  71eb45cb-0f34-4113-aa91-69c2d58863d2  40.745746 -73.903326  MTABC_7463   
11  8901a772-6372-41e0-b6f0-46542319fd95  40.745746 -73.903326  MTABC_7463   
12  32716878-40e7-4aaa-a3d8-87eac51f7d2b  40.745746 -73.903326  MTABC_7463   
13  ee29ceec-4420-4d61-92e8-c326f950b6b2  40.745746 -73.903326  MTABC_7463   
14  1c9cda6e-7dc4-4442-be61-b8453300502e  40.745746 -73.903326  MTABC_7463   
15  0050a6f8-e4bb-49a6-bff0-50bee36d587d  40.745786 -73.903307  MTABC_7463   
16  62f73123-a204-413b-9269-c3515701338d  40.745825 -73.903288  MTABC_7463   
17  b8488704-c119-48d7-ab95-0e6e77b33e37  40.745825 -73.903288  MTABC_7463   
18  4ac2c18e-68f4-4e76-a8d9-c23ce73e4814  40.745825 -73.903288  MTABC_7463   
19  4f7b041a-fc5b-4472-9e23-cae6df87a7d1  40.745786 -73.903307  MTABC_7463   
20  c3b3e304-bb8d-4457-b0d9-17117956aef4  40.745786 -73.903307  MTABC_7463   
21  b1957baf-e7b3-4757-bd03-46a4c321b744  40.745746 -73.903326  MTABC_7463   
22  1272eebc-2670-4deb-8e31-203818a42a79  40.745904 -73.903250  MTABC_7463   
23  a854b16b-8437-4cad-88c0-79cc31b60d9f  40.745786 -73.903307  MTABC_7463   
24  9144ad8a-dbd6-46bf-96d1-e61d2422681b  40.745688 -73.902816  MTABC_7463   
25  f72b9a7e-2f0a-4329-9e8f-1c50581aab8c  40.745688 -73.902816  MTABC_7463   
26  2c6f4a54-c964-42fb-b544-d3ebfad351fb  40.745786 -73.903307  MTABC_7463   
27  2426f36d-cc1a-42b2-a0ea-d340ccbae6c9  40.745825 -73.903288  MTABC_7463   
28  b6ad30e2-504f-40ed-8c3a-e0a9096cc9ce  40.745865 -73.903269  MTABC_7463   
29  30cba13b-71c1-407f-b096-fc3b620b5d43  40.745825 -73.903288  MTABC_7463   
30  58af615f-5094-436c-8a18-e7f9e6cce189  40.745825 -73.903288  MTABC_7463   
31  defc496d-f3a4-4d78-b4dc-944a66189fbe  40.745825 -73.903288  MTABC_7463   
32  5b9f52be-8373-4e64-8e21-4a3e7c18d4ad  40.745825 -73.903288  MTABC_7463   
33  ad8cff9b-b291-4a13-8405-656bcc445b88  40.745825 -73.903288  MTABC_7463   
34  1dace7c5-1785-4399-92ce-2fd9db8ccec0  40.745865 -73.903269  MTABC_7463   
35  fa78c7c2-ca48-496d-9b4e-68878196cd6f  40.745865 -73.903269  MTABC_7463   
36  f97fca96-d101-4acb-bdf5-d5060eb3f012  40.745865 -73.903269  MTABC_7463   
37  b93d93f3-0b38-45e5-ab48-26d4188bb5a1  40.745746 -73.903326  MTABC_7463   
38  5f992b06-88a9-4d26-8d55-ecc53a55d96f  40.745746 -73.903326  MTABC_7463   
39  333bb020-ecd8-44e5-9efc-518736f5a13c  40.745746 -73.903326  MTABC_7463   
40  54ae43b5-1d28-49b7-be86-7907071ed0ac  40.745746 -73.903326  MTABC_7463   
41  d2ccf6c8-db78-4e20-89b3-37dea5263d0f  40.745746 -73.903326  MTABC_7463   
42  f90b397c-d429-40dc-beae-4a78a577a5ab  40.745746 -73.903326  MTABC_7463   
43  92d3f9fe-948f-499f-b4df-75d4e96733a8  40.745746 -73.903326  MTABC_7463   
44  04af6501-4f74-4314-b8e7-dda92e33c11f  40.745746 -73.903326  MTABC_7463   
45  5a2c4303-0c51-4671-bf4c-623629849134  40.745746 -73.903326  MTABC_7463   
46  5d865a45-9959-495f-affa-d7a94578e3aa  40.745786 -73.903307  MTABC_7463   
47  c35cbf01-ff55-46f7-baa6-f5b4c53c02b3  40.745786 -73.903307  MTABC_7463   
48  a6e95b4e-3bed-49a6-b00f-51808d280df6  40.745786 -73.903307  MTABC_7463   
49  15eed8ee-781a-4429-b9fa-68b7affd6810  40.745786 -73.903307  MTABC_7463   
50  50ec298e-d358-4461-8df8-a53ac17b32a9  40.745786 -73.9033

In [39]:
dse51['time_delta'] = pd.to_timedelta(dse51['time_delta']).astype('timedelta64[s]')
dse51 = dse51.loc[(dse51['time_delta'] > 500) & (dse51['time_delta'] < 3600)]
#d51['delay'] = (d51['recorded_at_time'] - d51['next_predicted_arrival_time']).astype('timedelta64[s]')

dse49['time_delta'] = pd.to_timedelta(dse49['time_delta']).astype('timedelta64[s]')
dse49 = dse49.loc[(dse49['time_delta'] > 500) & (dse49['time_delta'] < 3600)]
#d49['delay'] = (d49['recorded_at_time'] - d49['next_predicted_arrival_time']).astype('timedelta64[s]')

dse48['time_delta'] = pd.to_timedelta(dse48['time_delta']).astype('timedelta64[s]')
dse48 = dse48.loc[(dse48['time_delta'] > 500) & (dse48['time_delta'] < 3600)]
#d48['delay'] = (d48['recorded_at_time'] - d48['next_predicted_arrival_time']).astype('timedelta64[s]')

dse50['time_delta'] = pd.to_timedelta(dse50['time_delta']).astype('timedelta64[s]')
dse50 = dse50.loc[(dse50['time_delta'] > 500) & (dse50['time_delta'] < 3600)]
#d50['delay'] = (d50['recorded_at_time'] - d50['next_predicted_arrival_time']).astype('timedelta64[s]')

dse53['time_delta'] = pd.to_timedelta(dse53['time_delta']).astype('timedelta64[s]')
dse53 = dse53.loc[(dse53['time_delta'] > 500) & (dse53['time_delta'] < 3600)]
#d53['delay'] = (d53['recorded_at_time'] - d53['next_predicted_arrival_time']).astype('timedelta64[s]')

dse46['time_delta'] = pd.to_timedelta(dse46['time_delta']).astype('timedelta64[s]')
dse46 = dse46.loc[(dse46['time_delta'] > 500) & (dse46['time_delta'] < 3600)]
#d46['delay'] = (d46['recorded_at_time'] - d46['next_predicted_arrival_time']).astype('timedelta64[s]')

dse45['time_delta'] = pd.to_timedelta(dse45['time_delta']).astype('timedelta64[s]')
dse45 = dse45.loc[(dse45['time_delta'] > 500) & (dse45['time_delta'] < 3600)]
#d45['delay'] = (d45['recorded_at_time'] - d45['next_predicted_arrival_time']).astype('timedelta64[s]')

dse72['time_delta'] = pd.to_timedelta(dse72['time_delta']).astype('timedelta64[s]')
dse72 = dse72.loc[(dse72['time_delta'] > 500) & (dse72['time_delta'] < 3600)]
#d72['delay'] = (d72['recorded_at_time'] - d72['next_predicted_arrival_time']).astype('timedelta64[s]')

dse55['time_delta'] = pd.to_timedelta(dse55['time_delta']).astype('timedelta64[s]')
dse55 = dse55.loc[(dse55['time_delta'] > 500) & (dse55['time_delta'] < 3600)]
#d55['delay'] = (d55['recorded_at_time'] - d55['next_predicted_arrival_time']).astype('timedelta64[s]')

dse63['time_delta'] = pd.to_timedelta(dse63['time_delta']).astype('timedelta64[s]')
dse63 = dse63.loc[(dse63['time_delta'] > 500) & (dse63['time_delta'] < 3600)]
#d63['delay'] = (d63['recorded_at_time'] - d63['next_predicted_arrival_time']).astype('timedelta64[s]')

In [40]:
dse45

id        lat        lon  vehicle_id  \
11   e7a047ab-1abd-48f8-89c2-d8ee9c0eca08  40.745746 -73.903326  MTABC_7445   
23   be17a3a0-bc44-4e7b-92dc-ac95115f2e17  40.745688 -73.902816  MTABC_7445   
40   e672dc65-f0d8-4fa7-9fbf-0b182941abf9  40.745825 -73.903288  MTABC_7445   
119  53f5bfac-1115-4a6c-8cac-7c01828df8be  40.745746 -73.903326  MTABC_7445   
126  7f0d796c-597c-4378-89f6-27c9728503b3  40.745688 -73.902816  MTABC_7445   
134  bebaf09c-6855-4d17-ae5d-baefbe941f34  40.745859 -73.903272  MTABC_7445   
138  1a004591-4077-46bd-bfd6-6ba9b553000e  40.745971 -73.903218  MTABC_7445   
140  87de9bf9-9756-4ac7-8a3b-da5477a00f43  40.745688 -73.902816  MTABC_7445   

    next_stop_id next_predicted_arrival_time next_predicted_departure_time  \
11    MTA_553332                         NaT                           NaN   
23    MTA_553332                         NaT                           NaN   
40    MTA_982075     2017-12-10 12:29:36.780    2017-12-10 12:29:36.780000   
119   MTA_553332                         NaT                           NaN   
126   MTA_553332                         NaT                           NaN   
134   MTA_982075                         NaT                           NaN   
138   MTA_982075     2017-12-10 19:06:18.106    2017-12-10 19:06:18.106000   
140   MTA_553332                         NaT                           NaN   

     next_stop_distance_from_origin    recorded_at_time  at_stop_boolean  \
11                             0.01 2017-12-10 10:47:45                1   
23                             0.01 2017-12-10 11:39:49                1   
40                         15252.51 2017-12-10 12:29:26                1   
119                            0.01 2017-12-10 13:58:27                1   
126                            0.01 2017-12-10 14:59:48                1   
134                        15252.51 2017-12-10 17:11:34                1   
138                        15252.51 2017-12-10 19:05:44                1   
140                            0.01 2017-12-10 19:49:59                1   

     time TIMEINT  time_delta  
11     10       2      2806.0  
23     11       2      2499.0  
40     12       2      2907.0  
119    13       3      3517.0  
126    14       3      3585.0  
134    17       3      2752.0  
138    19       4      2623.0  
140    19       4      3236.0

In [41]:
dsebus = pd.concat([dse51, dse49, dse48, dse50, dse53, dse46, dse45, dse72, dse55, dse63], ignore_index=True)

In [42]:
dsebus

id        lat        lon  vehicle_id  \
0   8b054d07-c9c8-4187-8570-00a02d5ea894  40.745688 -73.902816  MTABC_7451   
1   68bc5467-f0d9-402b-af34-e54768dc4600  40.745688 -73.902816  MTABC_7451   
2   6de36455-7efd-481b-876c-bfc12dc8ce38  40.745786 -73.903307  MTABC_7451   
3   961d719b-edf8-464b-926f-48b437274584  40.745894 -73.903255  MTABC_7451   
4   48b4d917-528e-4799-a58e-711ab7205f5f  40.745688 -73.902816  MTABC_7451   
5   25d709cd-89af-464a-91a8-add4316637bc  40.745746 -73.903326  MTABC_7451   
6   f23348cd-c18a-44cf-9f1e-06a75cb1f96a  40.745904 -73.903250  MTABC_7451   
7   8c0eadb7-c770-4ca2-803b-9dd0837b79fc  40.745688 -73.902816  MTABC_7451   
8   1a211a71-8871-436b-909c-832de250f090  40.745887 -73.903258  MTABC_7451   
9   1af2dd80-18a8-4a19-bea8-ba188d680983  40.745775 -73.903312  MTABC_7451   
10  d11f4c8d-eeca-472a-bd2c-40e6579afb8f  40.745982 -73.903213  MTABC_7449   
11  5e4dbf79-4836-41b9-9fda-1c65f1ecb87c  40.745746 -73.903326  MTABC_7449   
12  0c864a30-0e6f-4361-bdb7-8ca25e4b73fb  40.745796 -73.903302  MTABC_7449   
13  3733cedf-5775-4089-a70d-5f691de11037  40.745943 -73.903231  MTABC_7449   
14  f3835d4b-dc54-4482-8472-1f008adf0c62  40.745992 -73.903208  MTABC_7449   
15  22ed0d10-1937-4b8a-9bf8-35302d2c22f9  40.745786 -73.903307  MTABC_7449   
16  cd8fe04d-c35d-4841-86f5-e56507a6100f  40.745775 -73.903312  MTABC_7449   
17  70382650-8f96-4911-a074-7f7525c9a143  40.745688 -73.902816  MTABC_7448   
18  1e330036-c8b3-4a50-a4e6-5f1c024c5699  40.745688 -73.902816  MTABC_7448   
19  d689e644-d5b9-4253-b905-1386f91ec19d  40.745796 -73.903302  MTABC_7448   
20  97d4fdfc-e505-465a-bb4a-4a82ec7a5b4c  40.745796 -73.903302  MTABC_7448   
21  0527676e-eb3f-4ea4-b1c8-d8c920910a18  40.745796 -73.903302  MTABC_7448   
22  60bd1805-b2f7-4d7e-ad4e-3a3e4085d0fe  40.745845 -73.903278  MTABC_7448   
23  2fe35841-c074-4d26-aaad-4b9e8e448579  40.745786 -73.903307  MTABC_7448   
24  509f82bb-dcef-44df-bbe2-5a9bb121143b  40.745786 -73.903307  MTABC_7448   
25  09c5a59e-9a0b-4f54-9936-4d43690be9e7  40.745865 -73.903269  MTABC_7448   
26  3a37b4bc-5568-44dd-b6aa-214e336dde3c  40.745775 -73.903312  MTABC_7448   
27  732d43c4-958b-4acc-a7ca-a43ac6f90b19  40.745746 -73.903326  MTABC_7448   
28  e3e8e967-3100-4811-aef0-8b921fab9cb7  40.745688 -73.902816  MTABC_7448   
29  cd92a32e-a80a-42f4-9bfe-38de0646b1e7  40.745688 -73.902816  MTABC_7448   
..                                   ...        ...        ...         ...   
41  61091081-e42f-4809-9555-55f093cfdb25  40.745688 -73.902816  MTABC_7453   
42  a198b455-703b-4ea1-86c0-b8c365a6854a  40.745746 -73.903326  MTABC_7453   
43  56110cec-2318-45bc-99ca-723a2846421f  40.745887 -73.903258  MTABC_7453   
44  e5097a0f-5f8c-45f5-bc38-5dd9126d7683  40.745688 -73.902816  MTABC_7453   
45  b26547a9-99f0-41f8-a7db-0c09468717c1  40.745688 -73.902816  MTABC_7453   
46  5dfce0a6-043d-4701-9810-6d54134457eb  40.745688 -73.902816  MTABC_7453   
47  bae04e6d-2477-49ed-87dc-19c031a9d2e9  40.745943 -73.903231  MTABC_7453   
48  14c11a82-282c-42b8-8491-48fec59516c4  40.745786 -73.903307  MTABC_7453   
49  ec71025d-74ef-4c76-941e-162790f226ce  40.745786 -73.903307  MTABC_7446   
50  14e002e2-8d41-41b5-8f28-4723c972b021  40.745688 -73.902816  MTABC_7446   
51  0fd24240-7fab-4aeb-8b6d-a1d3b4b9483d  40.745688 -73.902816  MTABC_7446   
52  5e32570a-47fb-4787-b16e-24de3e078a72  40.745780 -73.903310  MTABC_7446   
53  e7a047ab-1abd-48f8-89c2-d8ee9c0eca08  40.745746 -73.903326  MTABC_7445   
54  be17a3a0-bc44-4e7b-92dc-ac95115f2e17  40.745688 -73.902816  MTABC_7445   
55  e672dc65-f0d8-4fa7-9fbf-0b182941abf9  40.745825 -73.903288  MTABC_7445   
56  53f5bfac-1115-4a6c-8cac-7c01828df8be  40.745746 -73.903326  MTABC_7445   
57  7f0d796c-597c-4378-89f6-27c9728503b3  40.745688 -73.902816  MTABC_7445   
58  bebaf09c-6855-4d17-ae5d-baefbe941f34  40.745859 -73.903272  MTABC_7445   
59  1a004591-4077-46bd-bfd6-6ba9b553000e  40.745971 -73.903218  MTABC_7445   
60  87de9bf9-9756-4ac7-8a3b-da5477a00f43  40.745688 -73.9028

# Calculate duration of route by time intervals

In [43]:
avg_time = dsebus.groupby(['TIMEINT']).mean()

In [44]:
avg_time = avg_time[['time_delta']].reset_index()

In [45]:
avg_time['Duration_in_mins'] = avg_time['time_delta'] / 60

In [46]:
avg_time

TIMEINT   time_delta  Duration_in_mins
0       1  2172.923077         36.215385
1       2  2664.772727         44.412879
2       3  3053.125000         50.885417
3       4  2599.250000         43.320833